In [ ]:
! pip install mistralai
! pip install datasets
from collections import defaultdict, Counter
import json
from matplotlib import pyplot as plt
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader

In [ ]:
from mistralai import Mistral

In [ ]:
from google.colab import userdata
userdata.get('MISTRAL_AI')
api_key = userdata.get("MISTRAL_AI")

In [ ]:
def run_mistral(user_message, model="mistral-large-latest"):
    client = Mistral(api_key=api_key)
    messages = [
        {"role":"user", "content":user_message}
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

# OpenAI summarized (hybrid) dataset
new = pd.read_csv('/content/gdrive/MyDrive/DSSI 2025/Group 1: DSSI Summer 2025/Data/summary_banktrak.csv')

# Split data
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(new, test_size=0.2, random_state=42) #validation size is 20%, random state is just there for reproducilbility

# Convert to dictionary
from datasets import Dataset, DatasetDict
dataset = DatasetDict({'train': Dataset.from_pandas(train_df, preserve_index=False), 'validation': Dataset.from_pandas(val_df, preserve_index=False)}) #use dataset dict to turn to dataframe to dict
dataset

df = pd.DataFrame(dataset["validation"][:])  # All validation data
output_path = '/content/gdrive/MyDrive/DSSI 2025/Group 1: DSSI Summer 2025/Data/summarized_validation_set.csv'
df.to_csv(output_path, index=False)

In [ ]:
# Give prompt
def user_message(inquiry):
    user_message = (
        f"""
        A Form 8-K is a public document that publicly traded companies must file with the Securities and Exchange Commission (SEC) to disclose significant, unscheduled events or corporate changes. These events are considered material, meaning they could impact the company's financial performance or investor decisions.
        Debt financing is when a company borrows money (or is given the right to borrow money) from a collection of lenders. This is primarily done through bonds, notes, loans, credit facilities. Key words include: Term Loan Agreement, Amended and Restated Credit Agreement, Revolving Credit Agreement, Junior Subordinated Notes, Bond, Debt Settlement Agreement.
        Label is 'True' if the item contains details about a debt instrument. If the item just references a debt instrument, this column should still be false. There must be some details outside of the name (such as the start date, the amount, the lenders, etc.).
        I want you to think like a financial analyst reviewing SEC Form 8-K filings to determine whether they contain information related to commercial debt. Your response should classify each text with a “True” or “False” label. Do not explain.

        ####
        Here are some examples:

        Text: On February 20,  2013, Washington Gas Energy Services, Inc (WGES), a subsidiary of WGL Holdings, Inc. (WGL Holdings), entered into a five-year secured supply arrangement with Shell Energy North America (US), LP (Shell Energy). WGES supplies electricity and natural  gas to residential, commercial and industrial customers in Maryland, Virginia, Delaware, Pennsylvania and the District of Columbia. Under this arrangement, WGES will have the ability to purchase the majority of its power, natural gas and related  products from Shell Energy in a structure that reduces WGES cash flow risk from collateral posting requirements. As a result of this credit support agreement, Shell Energy will have a lien on WGES assets; principally its retail accounts  receivables and contracts. A copy of the news release is attached as Exhibit 99.1.
        Label: FALSE
        Text: As previously disclosed, between December 14, 2022 and December 16, 2022, IronNet, Inc. (the " Company ") issued and sold secured promissory notes in an aggregate principal amount of $6.9 million (the " Initial Notes ") to a total of eight lenders, and on December 30, 2022, the Company issued a secured promissory note in the principal amount of $2.0 million (the " First   C5 Note ") to an entity affiliated with C5 Capital Limited (" C5 "), a beneficial owner of more than 5% of the Company's outstanding common stock.    In connection with the issuance of the First C5 Note, the Company and the holders of the Initial Notes agreed to amend and restate the Initial Notes to be substantially in the form to be issued to C5, which form was attached as an exhibit to the Current Report on Form  8-K  filed on January 4, 2023 (the " Restated Initial Notes "). This amendment and restatement occurred on January 11, 2023. The Restated Initial Notes continue to bear interest at a rate of 13.8% per annum from the respective dates of the Initial Notes, and the Restated Initial Notes are payable at scheduled maturity on June 30, 2023.    Upon entering into the First C5 Note, the Company and C5 also agreed to amend and restate the First C5 Note. Subsequently, the parties agreed that the amended and restated First C5 Note and any future notes to be issued in connection with additional bridge financing by C5 would be convertible into shares of the Company's common stock. On January 11, 2023, the Company issued to C5 an amended and restated convertible promissory note in the principal amount of $2.0 million (the " Restated C5 Note "). On January 12, 2023, the Company issued to C5 a new secured convertible promissory note in the principal amount of $3.0 million (the " Second C5 Note "). The Restated C5 Note and the Second C5 Note (collectively, the " C5 Notes "), in the aggregate principal amount of $5.0 million, are in substantially the form of secured convertible promissory note attached to this report as Exhibit 99.1.    The C5 Notes bear interest at a rate of 13.8% per annum from the date of issuance (or in the case of the Restated C5 Note, from the date of the First C5 Note), and all such notes are payable at scheduled maturity on June 30, 2023, subject to acceleration in certain circumstances.    The Company's obligations under the Restated Initial Notes and the C5 Notes are secured by substantially all of the assets of the Company, excluding the Company's intellectual property. C5 and each of the holders of the Restated Initial Notes have executed an Amended and Restated Security Agreement in the form attached as an exhibit to the Current Report on Form  8-K  filed on January 4, 2023.    The C5 Notes provide C5 with the right, at any time on or after the date that is five calendar days prior to maturity, to convert any portion of the principal amount of the C5 Notes, together with any accrued and unpaid interest and any other unpaid amounts, into shares of the Company's common stock, par value $0.0001 per share (the " Common Stock "), at a conversion price of $2.00 per share. In the event that any shares of Common Stock are issued upon conversion of the C5 Notes, the Company has agreed to grant specified registration rights to C5.    The foregoing description of the C5 Notes does not purport to be complete and is subject to, and is qualified in its entirety by reference to, the form of C5 Note filed herewith as Exhibit 99.1 and incorporated herein by reference.
        Label: TRUE
        Text: Credit Agreement  In connection with the completion of the Merger, on August 4, 2016, Higher One, Inc. ( Higher One ), a Delaware corporation and direct  wholly owned subsidiary of the Company, terminated the existing credit agreement, dated as of October 16, 2012 (as amended, the  Credit Agreement ), among Higher One, as borrower, the Company, Higher One Machines, Inc.,  Higher One Real Estate, Inc. and Higher One Real Estate SP, LLC, as guarantors, Bank of America, N.A. as administrative agent, swingline lender and letter of credit issuer, and the other lenders party thereto, and all commitments thereunder. At the  time of such termination, there were no outstanding revolving loans or outstanding letters of credit under the Credit Agreement. In connection with such termination, all liens on the assets of Higher One, the Company and Higher Ones  subsidiaries securing such facility, together with such subsidiary or Company guarantees, were released and terminated.
        Label: TRUE
        ###

        <<<
        Label: {inquiry}
        >>>
        """
    )
    return user_message

In [ ]:
# Testing on one text
print(run_mistral(user_message("""On January 13, 2023, the Company entered into a Term Loan Agreement (the "Agreement") with BigBank, LLP, as lender and administrative agent, providing for a term loan facility (the "Term Loan Facility") in an aggregate principal amount of up to $500 million. The Term Loan Facility will mature on January 13, 2028, and will bear interest at a rate of LIBOR plus 3.50% per annum. The Agreement contains customary covenants, events of default and remedies. The Company intends to use the net proceeds from the Term Loan Facility for general corporate purposes, including repayment of existing indebtedness.""")))

In [ ]:
# I want to extract the final label T/F
# reversed(raw_output.splitlines()): starts from the bottom of the output, since the label often appears last
# if "TRUE" in line.upper(): looks for any line containing "TRUE" (case-insensitive)
# returns a Python True or False accordingly

def run_mistral_clean(text):
    raw_output = run_mistral(user_message(text))

    # Option 1: Extract last line
    for line in reversed(raw_output.splitlines()):
        if "TRUE" in line.upper():
            return True
        if "FALSE" in line.upper():
            return False

    # Fallback
    raise ValueError(f"Unexpected output: {raw_output}")

In [ ]:
# Batch 1 with 11 texts
texts_1 = [
    "The information required by this Item 2.03 relating  to the Notes and the Indenture is contained in Item 1.01 above and is incorporated herein by reference.",
    "  On October 28, 2015, the Company disclosed the information set forth below.  Share Repurchase Program  The Company announced today the acceleration of its capital return plan with a $150 million share repurchase plan effective through the end of 2016 that was approved by the Companys Board of Directors.  The Company currently intends to repurchase its shares over time through a combination of the tender offer referred to below and open market purchases, and may also repurchase shares through private transactions, exchange offers, additional tender offers or other means. The Company intends to fund this program using its working capital.  Modified Dutch Auction Tender Offer  Under the new share repurchase program, the Company announced that it plans to launch promptly a modified Dutch auction tender offer to purchase up to $75 million of its common stock, at a price per share of not less than $8.50 and not greater than $9.25.  The tender offer will be contingent upon satisfaction of customary conditions. Additional details regarding the pricing and other terms will be provided upon formal commencement of the tender offer.  The Company had approximately 117.4 million shares of common stock outstanding as of October 28, 2015.  2  Information Regarding the Planned Tender Offer  This description is for informational purposes only and is not an offer to buy or the solicitation of an offer to sell any shares of the Companys common stock. The planned tender offer described in this Item 8.01 has not yet commenced, and there can be no assurance that the Company will commence the tender offer on the terms described in this Item 8.01 or at all. If the Company commences the tender offer, the tender offer will be made solely by an Offer to Purchase and the related Letter of Transmittal, as they may be amended or supplemented.  Stockholders and investors are urged to read any Tender Offer Statement on Schedule TO filed by the Company with the SEC in connection with the tender offer including the Offer to Purchase, the related Letter of Transmittal and other offer materials and exhibits thereto, as well as any amendments or supplements to the Schedule TO when they become available, because they will contain important information. If the Company commences the tender offer, it will file each of the documents referenced in this paragraph with the SEC, and, when available, investors may obtain them for free from the SEC at its website (www.sec.gov) or from the Companys information agent engaged in connection with the tender offer.  Note Regarding Share Repurchase Program  The Companys announcement of its share repurchase program does not obligate the Company to repurchase any specific dollar amount or number of its shares of common stock. The Company will determine when, if and how to proceed with any repurchase transactions under the program, as well as the amount of any such repurchase transactions, based upon, among other things, the results of the tender offer and the Companys evaluation of its liquidity and capital needs (including for strategic and other opportunities), its business, results of operations, and financial position and prospects, general financial, economic and market conditions, prevailing market prices for the Companys shares of common stock, corporate, regulatory and legal requirements, and other conditions and factors deemed relevant by the Companys management and Board of Directors from time to time. The share repurchase program may be suspended or discontinued at any time. There can be no assurance as to the actual volume of share repurchases in any given period or over the term of the program, if any, or as to the manner or terms of any such repurchases. The information in this Item 8.01 does not constitute an offer to sell or the solicitation of an offer to buy any securities in any jurisdiction.",
    " On August 16, 2021, Enstar Group Limited (the ""Company"") issued a press release announcing the commencement of a tender offer to purchase for cash any and all of its outstanding 4.500% Senior Notes due 2022 (the ""Tender Offer""). The Tender Offer is being conducted on the terms and conditions set forth in the offer to purchase dated August 16, 2021. The Tender Offer will expire at 5:00 p.m., New York City time, on Friday, August 20, 2021, unless extended or earlier terminated by the Company. A copy of the press release announcing the Tender Offer is attached as Exhibit 99.1 to this Current Report on Form 8-K and is incorporated by reference herein.",
    "Concho Resources Inc. and its subsidiary entered into a purchase agreement with Reliance to acquire oil and gas properties in the Northern Midland Basin for $1.625 billion. The properties have an estimated 10 MBoepd production and 43 MMBoe of proved reserves. The consideration for the acquisition includes cash and stock options. The company plans to finance the cash portion through offerings, credit facility, and cash on hand. The acquisition is expected to close in the second half of 2016, subject to customary conditions. The parties' obligations are dependent on various factors, including approvals and adjustments to the purchase price. The full Purchase Agreement is available for reference, but it is noted that the representations and warranties in the agreement may not reflect actual facts about Reliance or the Company, as they are used for risk allocation rather than factual statements.",
    "    Between December 14, 2022 and December 16, 2022, IronNet, Inc. (the "" Company "") issued and sold secured promissory notes in an aggregate principal amount of $6.9 million (the "" Notes "") to a total of eight lenders. The lenders included GEN Keith B. Alexander (Ret.), the founder, Chief Executive Officer and Chairman of the Board of Directors of the Company. The Notes were also issued to lenders that include VADM John M. McConnell (Ret.), Hon. Michael J. Rogers, Theodore E. Schlein and VADM Jan E. Tighe (Ret.), each of whom is a director of the Company, as well as funds affiliated with Forgepoint Capital (the "" Forgepoint Funds ""). Donald R. Dixon, a member of the Company's board of directors, is an affiliate of ForgePoint Capital, and the ForgePoint Funds are also beneficial owners of greater than 5% of the Company's common stock.    Each Note has a scheduled maturity date of one year from the date of issuance, except that the Note issued to one lender that is not, and is not affiliated with, a director of the Company has a scheduled maturity date of six months from the date of issuance. The Notes bear interest at a rate of 13.8% per annum, payable at maturity. The Company may prepay the Notes at any time without premium or penalty, and the Notes do not restrict the incurrence of future indebtedness by the Company.    In the event that the Company fails to timely pay amounts due under the Notes or the Company materially defaults in its performance of any other covenant under the Notes or the Security Agreement (as defined below), which default is not cured within 30 days after written notice thereof, then at the option of the lenders, upon the written consent of lenders representing a majority of the outstanding and unpaid principal under the Notes, all unpaid principal, accrued interest and other amounts owing under the Notes shall be immediately due and payable.    In the event that the Company files any petition or action for relief under any bankruptcy, reorganization, insolvency or moratorium law or any other law for the relief of or relating to debtors, or makes any assignment for the benefit of creditors or takes any corporate action in furtherance thereof, or if an involuntary petition is filed against the Company (unless such petition is dismissed or discharged within 60 days) under any bankruptcy statute, or a custodian, receiver, trustee, assignee for the benefit of creditors is appointed to take possession, custody or control of any property of the Company, then all unpaid principal, accrued interest and other amounts owing under the Notes will accelerate and automatically become immediately due and payable.    The Notes are secured by substantially all of the assets of the Company, excluding the Company's intellectual property. The Company entered into a Security Agreement (the "" Security Agreements "") with each of the lenders.    The forgoing descriptions of the Notes and the Security Agreements do not purport to be complete and are subject to, and are qualified in their entirety by reference to, the form of Note and form of Security Agreement filed hereto as Exhibits 4.1 and 10.1, respectively, and incorporated herein by reference.  ",
    "  On September 16, 2014, Enstar Group Limited (the Company), and certain of its subsidiaries, as borrowers and as guarantors, entered into a new  Revolving Credit Facility Agreement with National Australia Bank Limited (NAB), Barclays Bank PLC (Barclays), and Royal Bank of Canada (RBC), as mandated lead arrangers and original lenders, and NAB as agent (the  Credit Agreement).    The Credit Agreement provides for an unsecured five-year revolving credit facility (expiring in September 2019) pursuant  to which the Company is permitted to borrow up to an aggregate of $500 million (the Credit Facility), which is available to fund permitted acquisitions and for general corporate purposes. The Credit Agreement replaces and refinances the  Companys Prior Credit Agreement (as defined below). The Companys ability to draw on the Credit Facility is subject to customary conditions.  Interest is payable at the end of each interest period chosen by the Company or, at the latest, each six months. The interest rate is LIBOR plus a margin  factor initially set at 2.75%. The margin factor is subject to variation (ranging from 2.50% to 3.25%) in the event of a change to the Companys long term senior unsecured debt rating assigned by Standard & Poors Ratings Services  or Fitch Ratings Ltd. Any unused portion of the Credit Facility will be subject to a commitment fee of 35% of the applicable margin factor. The Credit Facility imposes various financial and business covenants on the Company and its subsidiaries,  including certain limitations on mergers and consolidations, acquisitions, indebtedness and guarantees, restrictions as to dispositions of stock and assets, and limitations on liens.  During the existence of any event of default (as specified in the Credit Agreement), the agent may cancel the commitments of the lenders, declare all or a  portion of outstanding amounts immediately due and payable or declare all or a portion of outstanding amounts payable upon demand. During the existence of any payment default, the interest rate would be increased by 1.0%. The Credit Facility  terminates and all amounts borrowed must be repaid on the fifth anniversary of the date of the Credit Agreement.    The foregoing description of the Credit  Agreement is qualified by reference to the full text of the Credit Agreement, which is filed as Exhibit 10.1 to this Current Report on Form 8-K.    The  Credit Agreement refinances and replaces, in its entirety, the Companys Revolving Credit Facility Agreement, originally dated June 14, 2011, as amended from time to time, and as amended and restated pursuant to the Restatement Agreement,  dated July 8, 2013, among the Company and certain of its subsidiaries, NAB and Barclays, as mandated lead arrangers, NAB, Barclays and RBC, as original lenders, and NAB as agent (the Prior Credit Agreement). The Prior Credit  Agreement had permitted the Company to borrow up to an aggregate of $375 million on a secured basis over a five-year term (the Prior Credit Facility). Effective September 16, 2014 and concurrent with its entry into the Credit  Agreement, the Company terminated the Prior Credit Agreement. Outstanding borrowings under the Prior Credit Facility totaled $319.6 million and were refinanced on September 16, 2014 with borrowings pursuant to the Credit Facility.  The Company was in compliance with all covenants under the Prior Credit Agreement and no material early termination fees were incurred in connection with the  termination.    The Prior Credit Facility had been secured by a first priority lien on the stock of certain of the Companys subsidiaries and certain  bank accounts held with Barclays in the name of the Company and into which amounts received in respect of any capital release from certain of the Companys subsidiaries were required to be paid. In connection with the termination of the Prior  Credit Agreement, all security pursuant to the Prior Credit Facility has been released, effective September 16, 2014.  ",
    "Item 2.03 incorporates the information from Item 1.01 regarding the Notes and the Note Purchase Agreement.",
    "Theravance, Inc. paid a cash dividend of $0.25 per share on June 30, 2015 to stockholders of record as of June 12, 2015. As a result, the conversion rate for the Company's unsecured convertible promissory notes due 2023 was adjusted from 48.9758 shares per $1,000 principal amount to 49.6963 shares per $1,000 principal amount, with an adjusted conversion price of approximately $20.1222 per share. This information is being disclosed but not filed for the Securities Exchange Act of 1934 or incorporated by reference in any filings under the Securities Act of 1933 or the Exchange Act.",
    "  Underwriting Agreement    On August 14, 2012, Concho Resources Inc. (the  Company ) and its subsidiaries entered into an underwriting agreement (the  Underwriting Agreement ) with J.P. Morgan  Securities LLC, as representative of the underwriters named in Schedule 1 thereto (collectively, the  Underwriters ), in connection with an underwritten public offering of $700 million aggregate principal amount of the  Companys 5.5% Senior Notes due 2023 (the  Notes ). The issuance and sale of the Notes has been registered under the Securities Act of 1933 (the  Securities Act ) pursuant to an automatic shelf Registration  Statement on Form S-3 (Registration No. 333-161809), as amended, of the Company, filed with the Securities and Exchange Commission (the  Commission ) on September 9, 2009 (the  Registration Statement ).  The Notes will be issued pursuant to a new supplemental indenture to the Companys existing indenture and will be guaranteed on a senior unsecured basis by certain of the Companys subsidiaries. Closing of the issuance and sale of the  Notes occurred on August 17, 2012. A legal opinion related to the Notes is filed herewith as Exhibit 5.1.    The  Underwriting Agreement contains customary representations, warranties and agreements by the Company and customary conditions to closing, obligations of the parties and termination provisions. Additionally, the Company has agreed to indemnify the  Underwriters against certain liabilities, including liabilities under the Securities Act, or to contribute to payments the Underwriters may be required to make because of any of those liabilities. Furthermore, the Company has agreed with the  Underwriters not to offer or sell any debt securities issued or guaranteed by the Company having a term of more than one year (other than the Notes) for a period of 45 days after the date of the Underwriting Agreement without the prior written  consent of J.P. Morgan Securities LLC.    The Underwriters or their affiliates have from time to time provided investment  banking, commercial banking and financial advisory services to the Company, for which they have received customary compensation. The Underwriters and their affiliates may provide similar services in the future. In particular, certain of the  underwriters or their affiliates are lenders under the Companys credit facility and will receive a portion of the net proceeds from the offering of the Notes. In addition, from time to time, certain of the Underwriters and their affiliates may  effect transactions for their own account or the account of customers, and hold on behalf of themselves or their customers, long or short positions in the Companys debt or equity securities or loans, and may do so in the future.  The foregoing description of the Underwriting Agreement is qualified in its entirety by reference to such Underwriting Agreement, a copy  of which is filed herewith as Exhibit 1.1 and is incorporated herein by reference.    Supplemental Indenture for 5.5%  Senior Notes due 2023    On August 17, 2012, the Company completed the public offering of the Notes. The Notes are  fully and unconditionally guaranteed on a senior unsecured basis by COG Acreage LP, COG Holdings LLC, COG Operating LLC, COG Production LLC, COG Realty LLC, Concho Oil & Gas LLC, Delaware River SWD LLC and Quail Ranch LLC (collectively, the  Subsidiary Guarantors ).  The terms of the Notes are governed by the indenture dated as of September 18, 2009  (the  Base Indenture ), among the Company, certain of the Subsidiary Guarantors, and Wells Fargo Bank, National Association, as trustee (the  Trustee ), as amended and supplemented by the seventh  supplemental indenture, dated as of August 17, 2012 (the  Supplemental Indenture ; the Base Indenture, as amended and supplemented by the Supplemental Indenture, the  Indenture ).  The Notes will mature on April 1, 2023, and interest is payable on the Notes on each April 1 and October 1, commencing on  October 1, 2012. The Company may redeem some or all of the Notes at any time on or after October 1, 2017 at the redemption prices specified in the Indenture. The Company may also redeem up to 35% of the Notes in an amount not greater than  the net proceeds of certain public sales of equity interests completed before October 1, 2015 at a redemption price as specified in the Indenture. If the Company sells certain assets or experiences specific kinds of change of control, each as  described in the Indenture, each holder of the Notes will have the right to require the Company to repurchase the Notes at a purchase price described in the Indenture plus accrued and unpaid interest, if any, to the date of repurchase.  The Notes are the Companys senior unsecured obligations, and will rank equally in right of payment with all of the Companys  existing and future senior debt, and will rank senior in right of payment to all of the Companys future subordinated debt. The Notes will be effectively subordinated to all of the Companys existing and future secured debt to the extent  of the value of the collateral securing such indebtedness.    The Indenture restricts the Companys ability and the ability  of certain of its subsidiaries to, among other things: (i) incur additional indebtedness; (ii) pay distributions or dividends on equity or purchase, redeem or otherwise acquire equity; (iii) make certain investments; (iv) use  assets as collateral in other transactions; (v) sell certain assets or merge with or into other companies; and (vi) enter into transactions with affiliates. These covenants are subject to a number of important exceptions and  qualifications.    The Indenture contains customary events of default, including:  default in any payment of interest on any Note when due, continued for 30 days;  default in the payment of principal of or premium, if any, on any Note when due;  failure by the Company to comply with its obligations under the Indenture, in certain cases subject to notice and grace periods;  payment defaults and accelerations with respect to other indebtedness of the Company and its Restricted Subsidiaries (as defined in the Indenture) in  the aggregate principal amount of $30.0 million or more;  certain events of bankruptcy, insolvency or reorganization of the Company or a Significant Subsidiary (as defined in the Indenture) or group of  Restricted Subsidiaries that, taken together, would constitute a Significant Subsidiary;  failure by the Company or any Significant Subsidiary or group of Restricted Subsidiaries that, taken together, would constitute a Significant  Subsidiary to pay certain final judgments aggregating in excess of $30.0 million within 60 days; and  any Subsidiary Guarantee of a Significant Subsidiary or group of Restricted Subsidiaries that, taken together, would constitute a Significant  Subsidiary, ceases to be in full force and effect, is declared null and void in a judicial proceeding or is denied or disaffirmed by its maker.  If an event of default under the Indenture occurs and is continuing, the Trustee or the  holders of at least 25% in principal amount of the outstanding Notes may declare the principal of, premium, if any, and accrued and unpaid interest, if any, on the Notes to be due and payable, or, in the case of certain events of default relating to  bankruptcy, insolvency or reorganization, those amounts will automatically become immediately due and payable.    Other material  terms of the Notes, the Base Indenture and the Supplemental Indenture are described in the final prospectus supplement, dated August 14, 2012, as filed by the Company and the Subsidiary Guarantors with the Commission on August 15, 2012.  The foregoing descriptions of the Indenture and the Notes are qualified in their entirety by reference to the Supplemental Indenture (including the form of Notes attached thereto), a copy of which is filed herewith as Exhibit 4.1 and is incorporated  herein by reference.  ",
    "The information required by this Item 2.03 relating to the Notes and the Indenture is contained in Item 1.01 above and is incorporated herein by reference.",
    " Enstar Group Limited (the ""Company""), and certain of its subsidiaries, as borrowers and as guarantors, are party to a five year revolving credit agreement dated August 16, 2018, most recently amended on December 19, 2018 (the ""Credit Agreement"") pursuant to which the Company is permitted to borrow up to an aggregate of $600 million (the ""Facility"") from time to time.  On March 31, 2020, the Company drew $150.0 million under the Facility, the proceeds of which were used to finance the Company's collateral and capital requirements relating to the transaction described below in Item 8.01 of this Current Report on Form 8-K. Following the drawdown, the Company has an aggregate amount outstanding under the Facility of $350.0 million, which includes a prior drawdown of $200.0 million on March 17, 2020 that the Company made to maximize financial flexibility in light of the uncertainty surrounding the impact of the coronavirus (COVID-19) pandemic. A description of the material terms of the Credit Agreement is included in the Company's  Current Report on Form 8-K  filed with the SEC on August 21, 2018 and is incorporated herein by reference, and such description is qualified in its entirety by the full text of the Credit Agreement, a copy of which was filed as  Exhibit 10.1  to the Form 8-K, as amended by the First Amendment to the Credit Agreement filed as  Exhibit 10.38  to the Company's Annual Report on Form 10-K for the year ended December 31,"
     ]
texts_1

true_labels_1 = [
    "FALSE",
    "FALSE",
    "TRUE",
    "FALSE",
    "TRUE",
    "TRUE",
    "FALSE",
    "FALSE",
    "TRUE",
    "FALSE",
    "TRUE"
]

In [ ]:
print(len(texts_1))
print(len(true_labels_1))

In [ ]:
# Batch 2 with 11 texts
texts_2 = [
    " On September 16, 2016, Washington Gas Light Company (the ""Company"") issued $250,000,000 aggregate principal amount of 3.796% Notes due 2046 (the ""Notes"") under its medium-term notes program, pursuant to a Terms Agreement (the ""Terms Agreement""), dated September 13, 2016, between the Company and Wells Fargo Securities, LLC, MUFG Securities Americas Inc., BB&T Securities, LLC, TD Securities (USA) LLC and The Williams Capital Group, L.P., as agents.  The Notes were offered to the public at par, and proceeds to the Company, net of commissions, were $248,125,000 (excluding expenses).  The Notes were registered under the Securities Act of 1933, as amended, pursuant to a shelf registration statement on Form S-3 (File No. 333-206576) filed by the Company.  As described in the Terms Agreement, the Notes are subject to prepayment at the Company's option at any time in whole or from time to time in part, at a redemption price equal to the greater of (i) 100% of the principal amount thereof and (ii) the sum of the present values of the remaining scheduled payments of principal and interest thereon, plus a make-whole call premium, plus, in either such case, accrued and unpaid interest on the principal of such Notes to the date of redemption.  At any time on and after March 15, 2046, the Company may redeem the Notes on any date or dates, in whole or from time to time in part, at 100% of the principal of such Notes, plus accrued and unpaid interest on the principal of such Notes to the date of redemption.  The Notes were issued pursuant to that certain Indenture, dated as of September 1, 1991, as amended by that certain Supplemental Indenture, dated as of September 1, 1993, each between the Company and The Bank of New York Mellon (formerly known as The Bank of New York), as trustee.",
    "On December 29, 2017, the Company sold its subsidiary Pavonia Holdings (US), Inc. to Southland National Holdings, Inc. for $120 million. The sale was completed through the Company's subsidiary Laguna Life Holdings SARL. The proceeds from the sale were used to repay debts under the Company's revolving credit facility. Pavonia Holdings owns Pavonia Life Insurance Company of Michigan and Enstar Life (US), Inc. A second closing is expected in the first or second quarter of 2018, where Southland will acquire Pavonia Life Insurance Company of New York for $13.1 million, subject to regulatory approval. This additional purchase price represents the cash consideration paid by Laguna to Pavonia Life Insurance Company of Michigan during the restructuring of the transaction.",
    "    The Company and C5 previously entered into an agreement pursuant to which they agreed to a mutual exclusivity period to seek to negotiate definitive agreements with respect to a potential offer by C5 to acquire all of the outstanding common stock of the Company not presently owned by C5 and certain of its affiliates. Upon receipt of the funds and issuance of the Fourth C5 Note by the Company, the parties agreed to extend the exclusivity period until March 14, 2023.  ",
    "  Underwriting Agreement  On September 30, 2015, Concho Resources Inc. (the  Company ) entered into an underwriting agreement (the  Underwriting Agreement ) with J.P. Morgan Securities LLC and Credit Suisse Securities (USA), LLC, as representatives (the  Representatives ) of the several underwriters named therein (the  Underwriters ), in connection with an underwritten public offering of up to 8,855,000 shares of common stock (the  Common Shares ), including the Underwriters option to purchase up to an  additional 1,155,000 Common Shares. On October 1, 2015, the Underwriters exercised in full their option to purchase additional Common Shares. The Company expects the net proceeds from the offering to be approximately $794 million, after deducting  estimated fees and expenses. Net proceeds from the offering are expected to be used to repay all outstanding borrowings under the Companys credit facility, which were used in part to finance recent acquisitions, and for general corporate  purposes, including funding potential future acquisitions. The issuance and sale of the Common Shares has been registered under the Securities Act of 1933, as amended (the  Securities Act ), pursuant to an automatic shelf  Registration Statement on Form S-3 (Registration No. 333-206172) of the Company, filed with and deemed automatically effective by the Securities and Exchange Commission on August 6, 2015. Closing of the issuance and sale of the Common  Shares is scheduled for October 6, 2015, subject to customary closing conditions. A legal opinion relating to the validity of the Common Shares is filed herewith as Exhibit 5.1.  The Underwriting Agreement contains customary representations, warranties and agreements by the Company and customary conditions to closing,  obligations of the parties and termination provisions. Additionally, the Company has agreed to indemnify the Underwriters against certain liabilities, including liabilities under the Securities Act, or to contribute to payments the Underwriters may  be required to make because of any of those liabilities. Furthermore, the Company has agreed with the Underwriters not to offer or sell any shares of its common stock (or securities convertible into or exchangeable for common stock), subject to  limited exceptions, for a period of 60 days after the date of the Underwriting Agreement without the prior written consent of the Representatives.  Certain of the Underwriters and their affiliates act as administrative agent, lender, swingline lender and/or letter of credit lender under  the Companys credit facility. Further, certain of the Underwriters or their affiliates have from time to time provided investment banking, commercial banking and financial advisory services to the Company, for which they have received  customary compensation. The Underwriters and their affiliates may provide similar services in the future. In addition, from time to time, certain of the Underwriters and their affiliates may effect transactions for their own account or the account  of customers, and hold on behalf of themselves or their customers, long or short positions in the Companys debt or equity securities or loans, and may do so in the future.  The foregoing description of the Underwriting Agreement is qualified in its entirety by reference to such Underwriting Agreement, a copy of  which is filed herewith as Exhibit 1.1 and is incorporated herein by reference.",
    "On January 17, 2013, Theravance announced the pricing of its offering of Notes through a press release. The following week, on January 22, 2013, Theravance issued another press release stating that the Underwriters had exercised their option to purchase additional Notes in full. Both press releases are attached as Exhibit 99.1 and Exhibit 99.2 respectively.",
    "The Form 8-K report revises the audited financial statements and management's discussion and analysis of Higher One Holdings, Inc. due to changes in the internal organization structure. The reportable segments have been retrospectively revised for prior periods. The changes are detailed in Note 17 of the financial statements. Higher One plans to sell assets and transfer liabilities related to the refund disbursement business, as outlined in the Asset Purchase Agreement with Customers Bank and Customers Bancorp, Inc. This will require the filing of proxy material including annual audited financial statements.",
    "    Securities Purchase Agreement and Convertible Note    On September 14, 2022, IronNet, Inc. (the "" Company "") entered into a Securities Purchase Agreement (the "" Purchase Agreement "") with an institutional investor (the "" Investor "") under which the Company agreed to sell and issue to the Investor senior unsecured convertible promissory notes (the "" Convertible Notes "") in the aggregate principal amount of up to $25,750,000. The Convertible Notes are convertible into shares of the Company's common stock, par value $0.0001 per share (the "" Common Stock ""), subject to certain conditions and limitations.    On September 15, 2022, the Company issued a Convertible Note to the Investor in the principal amount of $10,300,000. Upon the satisfaction of additional conditions set forth in the Purchase Agreement, the Company may issue an additional Convertible Note to the Investor in the principal amount of $15,450,000 at a second closing.    The Convertible Notes bear interest at an annual rate of 5.0% per annum, payable monthly on the first of each month (the "" Installment Date ""), beginning the first month that is 90 days following the issuance date of such Convertible Note, payable in cash and/or shares of Common Stock, at the Company's option. The interest rate will increase to an annual rate of 10.0% per annum upon the occurrence and during the continuance of an event of default under the Convertible Notes. Each Convertible Note issued pursuant to the Purchase Agreement will have a maturity date of 18 months from issuance, which may be extended at the option of the Investor in certain instances.    The Convertible Notes provide a conversion right, in which the Investor may, at any time after the issuance date, convert any portion of the principal amount of the Convertible Notes, together with any accrued and unpaid interest and any other unpaid amounts, into shares of Common Stock at a conversion price of $7.50 per share (the "" Conversion Price ""), subject to adjustment in accordance with the terms of the Convertible Notes. The Company shall not issue any shares of Common Stock upon conversion of any Convertible Notes, or otherwise, if the issuance of such Common Stock, together with any Common Stock issued in connection with the Purchase Agreement and the transaction contemplated thereby, would exceed 20,373,592 shares (the "" Exchange Cap ""), except that such limitation shall not apply in the event that the Company obtains the approval of its stockholders as required by the applicable rules of the NYSE for issuances of shares of Common Stock in excess of such amount.    On each monthly Installment Date for a Convertible Note, the Company shall repay the lesser of (i) $686,666.67, and (ii) the principal amount then outstanding under such Convertible Note, plus accrued and unpaid interest on the Convertible Note, in cash and/or shares of Common Stock, at the Company's option (collectively the "" Installment Amount ""). In certain instances, the Investor will also have the right to accelerate some of the monthly repayment obligations. For any Installment Amount paid in the form of shares of Common Stock, the applicable conversion price will be equal to the lesser of (a) the Conversion Price, and (b) the greater of (x) 95% of the lowest VWAP in the five trading days immediately prior to such conversion, and (y) a ""floor price"" of approximately $0.44 (equal to 20% of the closing price of the Common Stock on the NYSE on September 9, 2022), subject to adjustment in accordance with the terms of the Convertible Notes. For any Installment Amount paid in cash, the price paid will be equal to 105% of the Installment Amount.    If, at any time while any Convertible Note is outstanding, the Company or any of its subsidiaries shall carry out certain Common Stock and/or Common Stock equivalent financing transactions in excess of $5,000,000 in gross proceeds, the Investor shall have the right, subject to certain exceptions, to require the Company to first use up to 30% of the net proceeds of such financing to redeem all or a portion of such Convertible Note for an amount in cash equal to 105% of the sum of (i) the principal amount of such Convertible Note subject to such redemption, plus (ii) accrued but unpaid interest on the Convertible Note subject to such redemption, plus (iii) any other amounts, if any, then owing to the Investor in respect of such Convertible Note (the "" Redemption Price "").    In connection with a ""Change of Control"" (as defined in the Convertible Notes), the Investor shall have the right, subject to certain exceptions, to require the Company to redeem all or any portion of the Convertible Notes in cash at a price equal to the Redemption Price.  The Convertible Notes provide the Company, subject to certain conditions, with a redemption right pursuant to which the Company, upon 20 trading days' prior written notice to the Investor (a "" Redemption Notice ""), may redeem in cash at a price equal to the greater of (a) the Redemption Price, and (b) solely in certain instances the product of (x) the number of shares issued at the applicable Conversion Price with respect to the principal amount being redeemed, multiplied by (y) the greatest closing sale price of the Common Stock on any trading day during the period commencing on the date immediately preceding the date of such Redemption Notice and ending on the trading day immediately prior to the date the Company makes the voluntary redemption.    The Convertible Notes include customary covenants and set forth certain events of default after which the Convertible Notes may be declared immediately due and payable and set forth certain types of bankruptcy or insolvency events of default involving the Company after which the Convertible Notes become automatically due and payable. At any time after the earlier of (x) the Investor's receipt of a required notice of an event of default, and (y) the Investor becoming aware of an event of default, and ending on the tenth trading day after the later of (x) the date such event of default is cured, and (y) the Investor's receipt of an event of default notice, the Investor may require the Company to redeem, subject to certain exceptions, all or any portion of the Convertible Notes at a price equal to the greater of (a) 110% of the sum of (i) the principal amount subject to such redemption, plus (ii) accrued but unpaid interest on the Convertible Note, plus (iii) any other amounts, if any, then owing to the Investor in respect of such Convertible Note and (b) the product of (i) the number of shares issued at the applicable conversion price with respect to the principal amount being redeemed, multiplied by (ii) the product of (A) 105%, multiplied by (B) the greatest closing sale price of the Common Stock on any trading day during the period commencing on the date immediately preceding such event of default and ending on the date the Company makes such redemption.    Registration Rights Agreement    On September 14, 2022, in connection with the Company's entry into the Purchase Agreement, the Company also entered into a Registration Rights Agreement with the Investor (the "" Registration Rights Agreement ""). Pursuant to the Registration Rights Agreement, the Company has agreed to file with the Securities and Exchange Commission (the "" SEC ""), within 60 calendar days following the date of the Registration Rights Agreement, a registration statement covering the resale of the shares of Common Stock issuable upon conversion of the outstanding Convertible Note. Pursuant to the Registration Rights Agreement, the Company is required to use commercially reasonable efforts to have such registration statement declared effective by the SEC within a specified number of days.    The foregoing summaries of the terms of the Convertible Notes, the Purchase Agreement and the Registration Rights Agreement do not purport to be complete and are subject to, and qualified in their entirety by, the full text of the Convertible Note, which is attached as Exhibit 4.1 hereto and incorporated by reference herein, the full text of the Purchase Agreement, which is attached as Exhibit 10.1 hereto and incorporated by reference herein, and the full text of the Registration Rights Agreement, which is attached as Exhibit 10.2 hereto and incorporated by reference herein.",
    "Item 1.01 of the Current Report discusses Promissory Notes, which are further described in the Repurchase Agreements filed as Exhibits 10.1, 10.2, and 10.3. The full text of the Promissory Notes can be found in these exhibits.",
    "  Reliance Acquisition    On  August 15, 2016, Concho Resources Inc. (the  Company ) and its wholly owned subsidiary, COG Operating LLC, entered into a definitive purchase and sale agreement (the  Purchase Agreement ) with  Reliance Energy, Inc., Reliance Exploration, Ltd., Reliance Energy-WA, LLC, Reliance Energy-GF, LLC and the other persons named as sellers therein (collectively,  Reliance ) to acquire certain interests in oil and gas  properties, rights and related assets located in the Northern Midland Basin (the  Acquisition ), consisting of approximately 40,000 net acres for aggregate consideration of approximately $1.625 billion, subject to customary  purchase price adjustments and certain preferential rights to purchase. The properties have an average of 99% working interest (75% NRI) with minimal leasehold obligations. The properties to be acquired have an estimated current net production of  approximately 10 MBoepd from 44 horizontal and 326 vertical producing wells. Based on the estimates of the Companys internal reserve engineers, the Company estimates that there were approximately 43 MMBoe of proved reserves associated with  these properties as of June 30, 2016, of which 69% were proved developed. These reserve estimates were made in accordance with current SEC rules and regulations regarding oil and natural gas reserve reporting, including using average  first-day-of-the-month prices for the prior 12 months in accordance with SEC guidelines. The consideration for the Acquisition will consist of $1,137.5 million in cash and either an additional $487.5 million in cash or 3,958,526 shares of common  stock of the Company pursuant to an option exercisable by either the Company or Reliance to elect to require that a portion of the consideration be paid in common stock of the Company ( Stock Consideration ). The Company  anticipates exercising this option and paying a portion of the Acquisition consideration in common stock of the Company. If the election to pay a portion of the consideration as Stock Consideration is made, the Company has agreed pursuant to the  purchase agreement governing the Acquisition to file a registration statement to register the resale from time to time of the Stock Consideration under the Securities Act of 1933, as amended (the  Securities Act ), upon the  closing of the Acquisition.    The Company intends to finance the cash portion of the consideration for the Acquisition with proceeds from  its Offering (as defined below) as well as borrowings under its credit facility and cash on hand. The Company expects the Acquisition to close during the second half of 2016, subject to customary closing conditions, as further described below.  Each partys obligation to consummate the Acquisition is conditioned upon, among other things, (i) confirmation of the  counterparties representations and warranties as of the closing, (ii) the counterparties performance, in all material respects, of all covenants, (iii) the receipt of all required approvals (including approval under the  Hart-Scott-Rodino Antitrust Improvements Act of 1976), (iv) the absence of legal matters prohibiting the Acquisition, and (v) the aggregate value of adjustments to the purchase price attributable to title defects, environmental defects,  un-obtained consents to assignment, exercise of preferential purchase rights, the failure to consummate certain tag-along and drag-along rights, and casualty losses not exceeding an amount equal to 20% of the unadjusted purchase price.  A copy of the Purchase Agreement is filed as Exhibit 2.1 to this Current Report on Form 8-K and  is incorporated by reference herein. The above description of the Purchase Agreement is a summary only and is qualified in its entirety by reference to the complete text of the Purchase Agreement. The Purchase Agreement is filed herewith to provide  investors with information regarding its terms. It is not intended to provide any other factual information about Reliance or the Company. In particular, the assertions embodied in the representations and warranties contained in the Purchase  Agreement are qualified by information in confidential disclosure schedules provided by Reliance and the Company to each other in connection with the signing of the Purchase Agreement. These disclosure schedules contain information that modifies,  qualifies and creates exceptions to the representations and warranties set forth in the Purchase Agreement. Moreover, certain representations and warranties in the Purchase Agreement were used for the purpose of allocating risk between Reliance and  the Company rather than establishing matters of fact. Accordingly, you should not rely on the representations and warranties in the Purchase Agreement as characterizations of the actual statements of fact about Reliance or the Company.",
    "The company revised its segment structure in the first quarter of 2021 to align with the views of its Chief Executive Officer. The business is now organized into three reportable segments: Run-off, Investments, and Legacy Underwriting. The Current Report on Form 8-K presents financial results for the year ended December 31, 2020, consistent with the 2021 Quarterly Report. The report does not materially amend or restate audited consolidated financial statements from the 2020 Annual Report on Form 10-K. The information in the report reflects the revised segment structure and does not repeat unaffected items or portions of the 2020 Annual Report. The report should be read in conjunction with other filings made with the SEC after the 2020 Annual Report. Various sections of the 2020 Annual Report were revised to reflect changes in the segment structure. References to reportable segments were updated throughout the report, and a new section was added to address events that occurred after the filing of the 2020 Annual Report.",
    " As previously disclosed in our Quarterly Report on Form 10-Q for the quarter ended March 31, 2021 filed with the SEC on May 7, 2021 (the ""2021 Quarterly Report""), we revised our segment structure during the first quarter of 2021 to align with how our chief operating decision maker (who was determined to be our Chief Executive Officer) views our business, assesses performance and allocates resources to our business components. Effective January 1, 2021, following the completion of various strategic transactions related to both Atrium and StarStone, our business is now organized into three reportable segments: (i) Run-off; (ii) Investments; and (iii) Legacy Underwriting. This Current Report on Form 8-K presents our financial results for the year ended December 31, 2020 and is consistent with the presentation included in our 2021 Quarterly Report. The information included in this Current Report on Form 8-K, including Exhibit 99.2, is presented in connection with the revised segment structure described above and does not otherwise materially amend or restate our audited consolidated financial statements that were included in our 2020 Annual Report on Form 10-K. Unaffected items and unaffected portions of the 2020 Annual Report on Form 10-K have not been repeated in, and are not amended or modified by, this Current Report on Form 8-K, other than as required to reflect our updated reporting segments or as noted in the financial statements contained therein. Exhibit 99.2 to this Current Report on Form 8-K does not reflect events occurring after we filed the 2020 Annual Report on Form 10-K, except as otherwise noted therein. More current information is contained in the Company's filings subsequent to the date of filing of the 2020 Annual Report on Form 10-K, including the 2021 Quarterly Report, which also contains important information regarding forward-looking statements, events, developments or updates to certain expectations of the Company that have occurred since the filing of the 2020 Annual Report on Form 10-K. Accordingly, this Current Report on Form 8-K should be read in conjunction with our other filings made with the SEC, including, and subsequent to, the date of the 2020 Annual Report on Form 10-K. We have revised the following sections of the 2020 Annual Report on Form 10-K to reflect the retrospective revisions that have been made as a result of the changes to our segment structure: 1 * Part I, Item 1. Business: * Operating Segments * Liability for Losses and Loss Adjustment Expenses * Investments (incorporated into the Operating Segments discussion) * Part II, Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations: * Business Overview: # Non-GAAP Financial Measures # Underwriting Ratios (removed from this report) # Current Outlook * Consolidated Results of Operations - for the Years Ended December 31, 2020, 2019 and 2018 * Results of Operations by Segment - for the Years Ended December 31, 2020, 2019 and 2018 * Corporate and Other * Investable Assets * Liquidity and Capital Resources # Contractual Obligations # Critical Accounting Estimates * Part II, Item 8. Financial Statements and Supplementary Data: * Consolidated Financial Statements: # Report of Independent Registered Public Accounting Firm # Notes to the Consolidated Financial Statements: * Note 1 - Description of the Business * Note 8 - Reinsurance Balances Recoverable on Paid and Unpaid Losses * Note 10 - Losses and Loss Adjustment Expenses * Note 13 - Premiums Written and Earned * Note 14 - Goodwill and Intangible Assets * Note 24 - Segment Information * Schedules: # Schedule III - Supplementary Insurance Information In addition, references to reportable segments were also updated throughout Exhibit 99.2 to this Current Report on Form 8-K and we have added Note 26 - Events (Unaudited) Subsequent to the Date of the Independent Auditor's Report, to address updates that occurred after the filing of our 2020 Annual Report on 10-K."
]

true_labels_2 = [
    "TRUE",
    "FALSE",
    "FALSE",
    "FALSE",
    "FALSE",
    "FALSE",
    "TRUE",
    "FALSE",
    "FALSE",
    "FALSE",
    "FALSE"
]

In [ ]:
print(len(texts_2))
print(len(true_labels_2))

In [ ]:
# Batch 3 with 11 texts
texts_3 = [
    "Theravance, Inc. paid a cash dividend of $0.25 per share on March 31, 2015 to stockholders of record as of March 12, 2015. As a result, the conversion rate for the Company's unsecured convertible promissory notes due 2023 was adjusted from 48.3294 shares per $1,000 principal amount to 48.9758 shares per $1,000 principal amount. This adjustment represents an adjusted conversion price of approximately $20.4182 per share. The information provided is not considered filed under the Securities Exchange Act of 1934, nor is it subject to liabilities under that section. It is also not incorporated by reference in any filing under the Securities Act of 1933 or the Exchange Act, unless specifically referenced in such a filing.",
    " On March 14, 2018, the Company completed the public offering of the Notes pursuant to the Underwriting Agreement. The net proceeds of the offering of the Notes were approximately $248.8 million after deducting the underwriting discount and the Company's estimated offering expenses. The Company expects that the net proceeds from the offering will be used for general corporate purposes, including repayment of a portion of the amounts outstanding under the Company's unsecured revolving credit facility.  The offering of the Notes was made pursuant to a shelf registration statement on Form S-3 (File No. 333-220479), which became effective upon filing with the Securities and Exchange Commission (the ""SEC"") on September 15, 2017, including a prospectus, dated September 15, 2017, as supplemented by a prospectus supplement, dated March 12, 2018, which was filed with the SEC pursuant to Rule 424(b)(2) under the Securities Act.  The legal opinion of the Senior Vice President, General Counsel and Corporate Secretary of the Company relating to the offering of the Notes pursuant to the Registration Statement is filed as Exhibit 5.1 to this Current Report on Form 8-K.  The Company issued the Notes pursuant to the Indenture, dated as of November 29, 2017 (the ""Base Indenture""), as amended and supplemented by the Second Supplemental Indenture, dated as of March 14, 2018 (the ""Second Supplemental Indenture"" and, together with the Base Indenture, the ""Indenture""),  in each case by and between the Company and The Bank of New York Mellon, as trustee (the ""Trustee"").  Interest on the Notes will be paid quarterly in cash in arrears on March 12, June 12, September 12 and December 12 of each year, beginning on June 12, 2018, at a rate equal to the three-month London Interbank Offered Rate plus 55 basis points. The Notes will mature on March 12, 2020. The Notes are not redeemable.  The Indenture contains covenants, including that, subject to certain exceptions, the Company and its subsidiaries may not grant liens on any property of the Company or its subsidiaries, unless the Notes are secured by liens on an equal and ratable basis to those granted to such other creditors. The Company may issue additional debt securities ranking equally with, and otherwise similar in all respects to, the Notes (except for the public offering  price, the issue date and the first interest payment date, if applicable) so that those additional debt securities will be consolidated and form a single series with the Notes without the consent of the holders of the Notes.  The Trustee or the holders of at least 25% in principal amount of the Notes may require the Company to repay the entire principal amount of the Notes, including any premium and any interest, immediately upon the occurrence of the following events of default: (1) failure to pay any interest on the Notes within 30 days after the same becomes due and payable; (2) failure to pay the principal of or premium, if any, or any payment required by sinking or analogous fund, on the Notes when it becomes due and payable; and (3) failure to observe or perform any of our covenants or agreements applicable to the Notes for 60 days after notice of failure, either from the Trustee or from holders of at least 25% of the principal amount outstanding of the Notes. Upon the occurrence of certain events of receivership, insolvency, liquidation, bankruptcy or reorganization of the Company, all unpaid principal of, any premium and any accrued interest on the Notes will become automatically due and payable immediately, without any declaration or other act on the part of the Trustee or any holder.  The foregoing description does not purport to be complete and is qualified in its entirety by reference to the full text of each of the Base Indenture and the Second Supplemental Indenture, including the form of the Note, copies of which are attached or incorporated by reference herein as Exhibits 4.1, 4.2 and 4.3, respectively, to this Current Report on Form 8-K and are incorporated by reference herein.      ",
    "  On October 1, 2010, Enstar Group Limited (the Company) entered into share repurchase  agreements (the Repurchase Agreements) with three of its executives and certain trusts and a  corporation affiliated with them to repurchase an aggregate of 800,000 ordinary shares of the  Company at a price of $70.00 per share. The aggregate purchase price of $56,000,000 is payable by  the Company through promissory notes to each selling shareholder (each, a Promissory Note).  The Company agreed to repurchase an aggregate of 600,000 ordinary shares from Dominic F.  Silvester and R&H Trust Co. (NZ) Limited, as trustee of the Left Trust (the Left Trust). Mr.  Silvester is the Companys Chief Executive Officer and Chairman of the Board of Directors, and Mr.  Silvester and his immediate family are the sole beneficiaries of the Left Trust. Mr. Silvesters  beneficial ownership interest in the Company will be reduced from approximately 15.6% to 11.9% as a  result of the repurchase. The sales by Mr. Silvester are designed to reduce his holdings below 10%  after taking into consideration the 2,972,892 shares of nonvoting convertible ordinary shares of  the Company held by one of its subsidiaries (which shares are not considered to be outstanding for  financial reporting purposes). Mr. Silvester has informed the Company that he intends to become a  resident of the United States during 2011 or 2012, which will make Mr. Silvester a U.S. person  for purposes of the subpart F rules under the U.S. Internal Revenue Code. Under those rules, if  one or more U.S. persons that each own, directly or by attribution, 10% or more of a foreign  corporations outstanding shares, measured by voting power, own in the aggregate more than 25% of  the foreign corporations outstanding shares, by vote or value, the corporation will be a  controlled foreign corporation with respect to insurance income  which is generally defined  as insurance company income from insuring risks located outside its country of incorporation. In  that case, those U.S. shareholders must include in their taxable income each year their pro rata  share of the controlled foreign corporations insurance income even if that income is not  distributed. A substantial amount of the income normally earned by the Companys non-U.S.  subsidiaries that are insurance companies constitutes insurance income as so defined.  The Company believes that the nonvoting shares owned by its subsidiary do not affect the  determination whether the Company or any of its subsidiaries is a controlled foreign corporation,  and that the Companys bye-law restriction that caps any U.S. persons voting power with respect to  Company shares at 9.5% of the total voting power, coupled with pass-through voting for directors of  the Companys foreign subsidiaries, has operated and will continue to operate to preclude any U.S.  person from owning, directly or by attribution, more than 10%, measured by voting power, of the  shares of the Company or any of its foreign subsidiaries. Accordingly, the Company believes that  the ownership of 10% or more of its ordinary shares by a U.S. person should not cause the Company  or any of its subsidiaries to be controlled foreign corporations. Nonetheless, it is possible that  Internal Revenue Service might disagree with these positions. Therefore, the Company believes it  is prudent to avoid a circumstance in which U.S. persons that own 10% or more of its outstanding  shares, by vote or value, own in the aggregate more than 25% of its outstanding shares. Because  the Companys subsidiary that owns the nonvoting convertible ordinary shares is a U.S. person, any  negative tax consequences from the Company becoming a controlled foreign corporation might impact  that subsidiary  and, hence, the Company, as well as Mr. Silvester. By repurchasing the shares  from Mr. Silvester, the Company eliminates any such tax exposure that might otherwise have resulted  from his move to the United States.  The Company also agreed to repurchase 100,000 of its ordinary shares from R&H Trust Co. (BVI)  Limited, as trustee of the Elbow Trust (the Elbow Trust). Paul J. OShea (the Companys Joint  Chief Operating Officer, Executive Vice President and a member of its Board of Directors) and his  immediate family are the sole beneficiaries of the Elbow Trust. In addition, the Company agreed to  repurchase 100,000 of its ordinary shares from Hove Investments Holding Limited (Hove). Nicholas  A. Packer (the Companys Joint Chief Operating Officer and Executive Vice President) and his  immediate family are the sole beneficiaries of a trust that is the sole owner of Hove. While the  shares held by Messrs. OShea and Packer do not present the same tax issues as Mr. Silvesters  holdings, the Company believed it was fair and appropriate to offer each of them, as Mr.  Silvesters co-founders of the Company, the opportunity to sell shares back to the Company on the  same terms offered to Mr. Silvester.  Closing of the repurchase transactions is expected to occur as soon as practicable. At the  closing, the Company will issue a Promissory Note to Mr. Silvester in the principal amount of  $4,789,260 and to the Left Trust in the principal amount of $37,210,740. The Company will issue  Promissory Notes to both the Elbow Trust and  2  Hove in the principal amount of $7,000,000. The annual interest rate for the notes is fixed at  3.5% and the notes are repayable in three equal installments on December 31, 2010, December 1, 2011  and December 1, 2012. The holders may accelerate amounts payable under the Promissory Notes upon  the Companys default in payment, a sale of all or substantially all of the Companys assets,  formal action in contemplation of the Companys dissolution or liquidation or the institution of  bankruptcy proceedings. The Company may prepay the Promissory Notes in whole or in part at any  time without premium or penalty.  In connection with the Repurchase Agreements, the Company will enter into lock-up agreements  with each of Messrs. Silvester, OShea and Packer, the Left Trust, the Elbow Trust, Hove, and a  separate trust of which Mr. Silvester and his immediate family are the sole beneficiaries. The  lock-up agreements prohibit future sales and transfers of shares now owned or subsequently acquired  for two years from the date of the Repurchase Agreements.  The Repurchase Agreements and the transactions contemplated thereby were approved by the  Companys Audit Committee, which consists solely of its independent directors, on September 30,  2010. The closing price of the Companys ordinary shares on the NASDAQ Global Select Market on  September 29, 2010 was $72.11. The descriptions of the Repurchase Agreements are qualified in  their entirety by reference to the full text of the agreements, which are filed as Exhibits 10.1,  10.2 and 10.3 to this Current Report on Form 8-K and incorporated herein by reference.  In addition to the repurchases by the Company, Messrs. OShea and Packer have also entered  into agreements to sell 32,500 shares each at $70.00 per share to certain investment funds  affiliated with Akre Capital Management, LLC. Charles T. Akre, a member of the Companys Board of  Directors, is the managing member of Akre Capital Management, LLC. Mr. OSheas beneficial  ownership interest in the Company will be reduced from approximately 4.6% to 3.9% as a result of  the two transactions. Mr. Packers beneficial ownership interest in the Company will be reduced  from approximately 4.4% to 3.7% as a result of the two transactions.",
    "In December 2020, Innoviva, Inc. partnered with Sarissa Capital Management LP to enhance its capital returns and acquisition strategy. The partnership includes a strategic advisory agreement where Sarissa Capital will assist Innoviva in developing an acquisition strategy and an investment by Innoviva into an investment fund managed by Sarissa Capital. The partnership aims to increase shareholder value by diversifying Innoviva's business and optimizing capital allocation. Sarissa Capital will provide strategic services to assist in the development and execution of Innoviva's acquisition strategy. Innoviva also made a $300 million initial contribution to a partnership with Sarissa Capital, focusing on investments in the healthcare industry. Sarissa Capital will receive a management fee and performance allocation based on the Partnership's profits. The Partnership Agreement includes a lock-up period, withdrawal rights, and distribution provisions. Strategic Partners can also receive annual distributions of profits and has co-investment and capacity rights. The partnership was approved by a special committee of independent directors, with two directors recusing themselves due to their affiliation with Sarissa Capital. The terms of the partnership and investment agreements include customary provisions such as indemnities, information rights, and reporting obligations.",
    "On August 26, 2020, Enstar Group Limited (the ""Company"") and its wholly-owned subsidiary, Enstar Finance LLC (""Enstar Finance""), completed a previously-announced underwritten public offering (the ""Offering"") of $350 million aggregate principal amount of 5.750% Fixed-Rate Reset Junior Subordinated Notes due 2040 (the ""Junior Subordinated Notes""). The Junior Subordinated Notes were issued by Enstar Finance and are fully and unconditionally guaranteed, on a junior subordinated basis, by the Company. In connection with the Offering, on August 26, 2020, the Company, Enstar Finance and The Bank of New York Mellon, as trustee (the ""Trustee""), entered into a first supplemental indenture (the ""First Supplemental Indenture"") to the junior subordinated indenture, dated as of August 26, 2020 (the ""Junior Subordinated Indenture""). The Junior Subordinated Indenture and the First Supplemental Indenture set forth the terms and conditions of the Junior Subordinated Notes, including customary events of default and the rights and obligations of the parties thereto and the holders of the Junior Subordinated Notes. Copies of the Junior Subordinated Indenture and the First Supplemental Indenture are filed as Exhibits 4.1 and 4.2 hereto, respectively, and are incorporated herein by reference. The Junior Subordinated Notes will bear interest (a) from the date of original issue to, but excluding, September 1, 2025 at the fixed rate of 5.750% per annum and (b) from, and including, September 1, 2025, during each five-year period thereafter, at a rate per annum equal to the Five-Year Treasury Rate as of two business days prior to the beginning of such five-year period plus 5.468% as reset at the beginning of each such five-year period. Interest will be paid on September 1 and March 1 of each year, commencing on March 1, 2021. If, as of any interest payment date, a Mandatory Deferral Event (as defined below) has occurred and is continuing, Enstar Finance and the Company will be required to defer payment of all (and not less than all) of the interest accrued on the Junior Subordinated Notes as of such interest payment date. A ""Mandatory Deferral Event"" will be deemed to have occurred if the Company or all of its subsidiaries that are regulated insurance or reinsurance companies (or part of such regulatory group) are in breach of the enhanced capital and surplus requirements under applicable insurance supervisory laws (the ""Enhanced Capital Requirements""), or would breach such Enhanced Capital Requirement if payment of accrued and unpaid interest on the Junior Subordinated Notes, together with any accrued and unpaid interest on any junior subordinated notes outstanding that rank equally in right of payment with the Junior Subordinated Notes, were made. The Junior Subordinated Notes are scheduled to mature on September 1, 2040, if, on such date, certain redemption requirements are satisfied, or otherwise, following such scheduled maturity date, the earlier of (a) the date falling ten business days after the redemption requirements are satisfied and would continue to be satisfied if such payment were made and (b) the date on which a winding-up of the Company or Enstar Finance occurs (the ""Final Maturity Date""). The Junior Subordinated Notes may not be repaid prior to the Final Maturity Date if the Enhanced Capital Requirement would be breached immediately before or after giving effect to such repayment of the Junior Subordinated Notes, unless the Company, Enstar Finance or another subsidiary of the Company replaces the capital represented by the Junior Subordinated Notes to be repaid with capital having equal or better capital treatment as the Junior Subordinated Notes under applicable insurance supervisory laws. The Junior Subordinated Indenture and the First Supplemental Indenture provide for the following events of default (subject in certain cases to customary grace and cure periods): nonpayment of principal or any premium when due (other than if Enstar Finance is required to postpone payment to satisfy certain redemption requirements); nonpayment of interest (except if due to a Mandatory Deferral Event); the Company's guarantee of the Junior Subordinated Notes ceases to be in full force and effect (other than a permitted release of the guarantee); or certain events of bankruptcy, insolvency or reorganization of Enstar Finance or the Company. Generally, if an event of default occurs, the trustee or the holders of at least 25% in aggregate principal amount of the then outstanding Junior Subordinated Notes may declare the principal and accrued and unpaid interest on all of the Junior Subordinated Notes to be due and payable immediately. 1 The foregoing descriptions of the Junior Subordinated Indenture and the First Supplemental Indenture are qualified by reference to the agreements themselves, which are attached as exhibits to this report.",
    "    Notwithstanding the receipt of proceeds from the issuance of the Notes and other cost reduction measures, the Company continues to actively pursue additional financing, and the Board of Directors has initiated a review of strategic alternatives for the Company, including retaining advisors to assist in the strategic review process.    The Company has used the proceeds of the Notes to satisfy certain of its current liabilities. However, even after the application of the proceeds, management anticipates that, in the absence of additional sources of liquidity, the Company's existing cash and cash equivalents and anticipated cash flows from operations will not be sufficient to meet the Company's operating and liquidity needs beyond the end of December 2022.    In the event the Company determines that additional sources of liquidity will not be available to it or will not allow it to meet its obligations as they become due, the Company may need to file a voluntary petition for relief under the United States Bankruptcy Code in order to implement a plan of reorganization, court-supervised sale and/or  liquidation. Furthermore, in the event the Company is unable to pursue bankruptcy protection under Chapter 11 of the United States Bankruptcy Code, it may be necessary to pursue bankruptcy protection under Chapter 7 of the United States Bankruptcy Code, in which case a Chapter 7 trustee would be appointed or elected to liquidate the Company's assets for distribution in accordance with the priorities established by the United States Bankruptcy Code. The Company expects that liquidation under Chapter 7 would result in significantly smaller distributions being made to stakeholders than those it might obtain under Chapter 11 primarily because of the likelihood that the Company's assets would have to be sold or otherwise disposed of by a Chapter 7 trustee in a distressed fashion over a short period of time rather than sold by existing management as a going concern business. In the event that the Company pursues bankruptcy protection under Chapter 7, the Company's material business activities will cease, and the Company will no longer have the capability to prepare financial statements and other disclosures required for periodic reports for filing with the Securities and Exchange Commission. The Company expects that no distributions would be available for stockholders in a Chapter 7 liquidation.    Forward-Looking Statements    Certain statements in this Current Report on Form  8-K  may be considered forward-looking statements, including statements with respect to the Company's expectations with respect to the sufficiency of the Company's cash resources and the ability to secure additional sources of liquidity, the Company's ability to continue as a going concern and the potential need to pursue bankruptcy protection. Forward-looking statements generally relate to future events and can be identified by terminology such as ""may,"" ""should,"" ""could,"" ""might,"" ""plan,"" ""possible,"" ""strive,"" ""budget,"" ""expect,"" ""intend,"" ""will,"" ""estimate,"" ""believe,"" ""predict,"" ""potential,"" ""pursue,"" ""aim,"" ""goal,"" ""mission,"" ""anticipate"" or ""continue,"" or the negatives of these terms or variations of them or similar terminology. Such forward-looking statements are subject to risks, uncertainties, and other factors which could cause actual results to differ materially from those expressed or implied by such forward-looking statements. These forward-looking statements are based upon estimates and assumptions that, while considered reasonable by the Company and its management, are inherently uncertain. Factors that may cause actual results to differ materially from current expectations include, but are not limited to, risks and uncertainties set forth in the section entitled ""Risk Factors"" in the Company's Quarterly Report on Form  10-Q  for the fiscal quarter ended July 31, 2022 and in the sections entitled ""Risk Factors"" and ""Cautionary Note Regarding Forward-Looking Statements"" in the Company's Annual Report on  Form 10-K for the  fiscal year ended January 31, 2022, and other documents filed by the Company from time to time with the SEC. These filings identify and address other important risks and uncertainties that could cause actual events and results to differ materially from those contained in the forward-looking statements. Forward-looking statements speak only as of the date they are made. Readers are cautioned not to put undue reliance on forward-looking statements, and the Company assumes no obligation and does not intend to update or revise these forward-looking statements other than as required by applicable law. The Company does not give any assurance that it will achieve its expectations.  ",
    " On June 28, 2018, Fuse Enterprises Inc. (the ""Company"") entered into a Share Purchase Agreement (the ""Purchase Agreement"") with Fuse Trading Limited, a wholly-owned subsidiary of the Company (""Trading""), and Landbond Home Limited (the ""Purchaser""), a company incorporated under the laws of Samoa and a major shareholder of the Company, pursuant to which the Company agreed to sell to the Purchaser in a private placement (the ""Private Placement"") 19,628,050 shares (the ""Shares"") of the Company's common stock, par value $0.001 per share, at a purchase price of $0.35 per Share for an aggregate offering price of $6,869,817.60 (the ""Purchase Price""). The Purchaser will pay the Purchase Price through the cancellation of a promissory note issued by Trading to the Purchaser in the amount of the Purchase Price. The Private Placement will be completed pursuant to the exemption from registration provided by Regulation S promulgated under the Securities Act of 1933, as amended, and the Company will issue the shares to the Purchaser's designees.  The representations, warranties and covenants contained in the Agreement were made solely for the benefit of the parties to the Agreement. In addition, such representations, warranties and covenants (i) are intended as a way of allocating the risk between the parties to the Agreement and not as statements of fact, and (ii) may apply standards of materiality in a way that is different from what may be viewed as material by shareholders of, or other investors in, the Company. Accordingly, the Agreement is filed with this report only to provide investors with information regarding the terms of transactions, and not to provide investors with any other factual information regarding the Company. Shareholders should not rely on the representations, warranties and covenants or any descriptions thereof as characterizations of the actual state of facts or condition of the Company. Moreover, information concerning the subject matter of the representations and warranties may change after the date of the Agreement, which subsequent information may or may not be fully reflected in public disclosures.  The Agreement is filed as Exhibit 10.1 to this Current Report on Form 8-K. The foregoing summary of the terms of the Agreement is subject to, and qualified in its entirety by, the Agreement, which is incorporated herein by reference. ",
    " The conversion rate adjustment with respect to our 2.125% Convertible Subordinated Notes due 2023 (the Notes) that results from our spin-off of Theravance Biopharma, Inc. (Theravance Biopharma) is based on the ten consecutive trading day period that ended on June 16, 2014 (the Spin-Off Valuation Period).   Based on the average of the closing prices of our common stock and of Theravance Biopharma ordinary shares over the Spin-Off Valuation Period, the conversion rate of 46.9087 shares of our common stock for each $1,000 principal amount of Notes will become effective on June 17, 2014.  Prior to the spin-off, the conversion rate for the Notes was 35.9903 shares of our common stock for each $1,000 principal amount of Notes.",
    " On August 5, 2016, Enstar Group Limited (the ""Company""), and certain of its subsidiaries, as Borrowers and as Guarantors, entered into an amendment and restatement of the Company's existing unsecured $665 million revolving credit facility agreement (the ""Restatement Agreement"") with National Australia Bank Limited (""NAB""), Barclays Bank PLC (""Barclays""), Lloyds Bank plc (""Lloyds Bank"") and SunTrust Bank (""SunTrust"") as Lenders; NAB, Barclays, Lloyds Bank and SunTrust Robinson Humphrey, Inc. as Mandated Lead Arrangers; NAB as Agent and Royal Bank of Canada (""RBC""). The Restatement Agreement provides for the restatement of the existing revolving credit facility agreement primarily in order to: (1) reduce the applicable margin factor, (2) permit the Company to provide guarantees in the ordinary course of its insurance business without prior approval from the lenders, (3) provide for a utilization fee on amounts drawn, (4) add SunTrust as a new lender within the facility, replacing RBC and (5) allow the Company to obtain additional commitments of up to $166.25 million, on and subject to the terms of a restated facility agreement (the ""Restated Facility Agreement""). The Restated Facility Agreement provides for an unsecured five-year revolving credit facility (expiring in September 2019) pursuant to which the Company is permitted to borrow up to an aggregate of $665.0 million (the ""Credit Facility""), which is available to fund permitted acquisitions and for general corporate purposes. Each of the lenders under the Restated Facility Agreement has agreed to provide an equal amount of $166.25 million to fund the Credit Facility, and pursuant to the Restated Facility Agreement, the Company has the option to obtain additional commitments under the Credit Facility up to an aggregate of $166.25 from the existing lenders or through the addition of one or more new lenders subject to the terms of the agreement. Interest is payable at the end of each interest period chosen by the Company or, at the latest, each six months. The interest rate is LIBOR plus a margin factor, which is subject to variation in the event of a change to the Company's long term senior unsecured debt rating assigned by Standard & Poor's Ratings Services or Fitch Ratings Ltd. Under the Restated Facility Agreement, the Company will pay usual and customary commitment and utilization fees that vary based on the aggregate amount of loans outstanding.  The Restated Facility Agreement imposes various financial and business covenants on the Company and its subsidiaries, including certain limitations on mergers and consolidations, acquisitions, indebtedness and guarantees, restrictions as to dispositions of stock and assets, and limitations on liens. The Restated Facility Agreement permits the Company to enter into guarantees during the ordinary course of its insurance business without prior approval from the lenders. During the existence of any event of default (as specified in the Restated Facility Agreement), the agent may cancel the commitments of the lenders, declare all or a portion of outstanding amounts immediately due and payable or declare all or a portion of outstanding amounts payable upon demand. In the event of a payment default, interest would accrue on the unpaid amount from the due date up to the date of actual payment at a rate that is 1.0% higher than the interest rate then in effect. The Credit Facility terminates and all amounts borrowed must be repaid on the fifth anniversary of September 16, 2014, the date on which the Credit Facility was originally entered into. The foregoing description of the Restatement Agreement, including the description of the Restated Facility Agreement attached as a schedule thereto, is qualified by reference to the full text of the Restatement Agreement, and the Restated Facility Agreement attached as a schedule thereto, filed as Exhibit 10.1 to this Current Report on Form 8-K",
    "  Supplemental Indentures for 4.300% Senior Notes due 2028 and 4.850% Senior Notes due 2048  On July 2, 2018, Concho Resources Inc. (the  Company ) completed the public offering of $1,000 million aggregate principal  amount of the Companys 4.300% Senior Notes due 2028 (the  2028   notes ) and $600 million aggregate principal amount of the Companys 4.850% Senior Notes due 2048 (the  2048  notes  and, together with the 2028 notes, the  Notes ). The Notes are fully and unconditionally guaranteed on a senior unsecured basis by certain of the Companys current subsidiaries (collectively, the  Subsidiary Guarantors ).    The terms of the Notes are governed by the indenture dated as of September 18, 2009 (the  Base Indenture ), among the Company, certain of the Subsidiary Guarantors, and Wells Fargo Bank, National Association, as trustee (the  Trustee ), as amended and supplemented by, in the case of the  2028 notes, the fourteenth supplemental indenture, dated as of July 2, 2018 (the  Fourteenth   Supplemental Indenture ) and by, in the case of the 2048 notes, the fifteenth supplemental indenture, dated as of  July 2, 2018 (the  Fifteenth Supplemental Indenture ; the Base Indenture, as amended and supplemented by the Fourteenth Supplemental Indenture and the Fifteenth Supplemental Indenture, the  Indenture ).    The 2028 notes will mature on August 15, 2028, and interest is payable on the 2028 notes on each  August 15 and February 15, commencing on February 15, 2019. The Company may redeem some or all of the 2028 notes at any time on or after May 15, 2028 at par, and it may redeem some or all of the 2028 notes at any time prior to  May 15, 2028 on a make-whole basis, in each case as specified in the Fourteenth Supplemental Indenture. The 2048 notes will mature on August 15, 2048, and interest is payable on the 2048 notes on each August 15 and  February 15, commencing on February 15, 2019. The Company may redeem some or all of the 2048 notes at any time on or after February 15, 2048 at par, and it may redeem some or all of the 2048 notes at any time prior to  February 15, 2048 on a make-whole basis, in each case as specified in the Fifteenth Supplemental Indenture. If a Change of Control Triggering Event (as defined in the Indenture) occurs with respect to either series of the Notes,  each holder of the Notes of that series will have the right to require the Company to repurchase such holders Notes at a purchase price equal to 101% of par, plus accrued and unpaid interest, if any, to the date of repurchase.  The Notes are the Companys senior unsecured obligations and will rank equally in right of payment with all of the Companys existing and future  senior debt, including all borrowings under the Companys credit facility, and rank senior in right of payment to all of the Companys future subordinated debt. The Notes will be effectively subordinated in right of payment to all of the  Companys future secured debt to the extent of the value of the collateral securing such indebtedness and will be structurally subordinated to all liabilities of any of the Companys future subsidiaries that do not guarantee the Notes.  The Indenture restricts the Companys ability and the ability of certain of its subsidiaries to, among other things: (i) create liens that  secure debt and (ii) merge or consolidate with another company. These covenants are subject to a number of important exceptions and qualifications.  The Indenture contains customary events of default with respect to the Notes of either series, including:  default in any payment of interest on any Note of that series when due, continued for 30 days;  default in the payment of principal of or premium, if any, on any Note of that series when due;  failure by the Company to comply with its other obligations under the Indenture, in certain cases subject to notice and grace periods;  payment defaults and accelerations with respect to other indebtedness of the Company and its Subsidiaries (as defined in the Indenture) in the aggregate principal amount of $150.0 million or more;  certain events of bankruptcy, insolvency or reorganization of the Company or a Significant Subsidiary (as defined in the Indenture) or group of Subsidiaries that, taken together, would constitute a Significant  Subsidiary;  failure by the Company or any Significant Subsidiary or group of Subsidiaries that, taken together, would constitute a Significant Subsidiary to pay certain final judgments aggregating in excess of $150.0 million  within 60 days; and  any Subsidiary Guarantee of a Significant Subsidiary or group of Subsidiaries that, taken together, would constitute a Significant Subsidiary, ceases to be in full force and effect, is declared null and void in a  judicial proceeding or is denied or disaffirmed by its maker.    If an event of default under the Indenture occurs with respect to either  series of the Notes and is continuing, the Trustee or the holders of at least 25% in principal amount of the outstanding Notes of the applicable series may declare the principal of, premium, if any, and accrued and unpaid interest, if any, on the  Notes of that series to be due and payable, or in the case of certain events of default relating to bankruptcy, insolvency or reorganization, those amounts will automatically become immediately due and payable.  Other material terms of the Notes, the Base Indenture, the Fourteenth Supplemental Indenture and the Fifteenth Supplemental Indenture are described in the  final prospectus supplement, dated June 14, 2018, as filed by the Company and the Subsidiary Guarantors with the Securities and Exchange Commission on June 15, 2018. The foregoing descriptions of the Indenture and the Notes are qualified  in their entirety by reference to the Fourteenth Supplemental Indenture and the Fifteenth Supplemental Indenture (including, in each case, the form of Notes attached thereto), copies of which are filed herewith as Exhibit 4.1 and Exhibit 4.2 and are  incorporated herein by reference.  ",
    "In connection with its entry in the Credit Agreement described above, on August 17, 2018, the Company terminated the amended and restated revolving credit facility agreement dated August 5, 2016 (the ""Prior Credit Agreement""), which provided for a $600 million unsecured revolving credit facility that originated on September 16, 2014 among the Company and certain of its subsidiaries, as borrowers and as guarantors; NAB, Barclays, Lloyds Bank plc (""Lloyds Bank"") and SunTrust Bank (""SunTrust"") as lenders; NAB, Barclays, Lloyds Bank and SunTrust Robinson Humphrey, Inc. as mandated lead arrangers; NAB as agent and Royal Bank of Canada. For a complete description of the material terms of the Prior Credit Agreement, refer to Item 1.01 of the Company's Current Report on Form 8-K filed on August 11, 2016, which is incorporated by reference into this item 1.02."
   ]

true_labels_3 = [
    "FALSE",
    "TRUE",
    "TRUE",
    "FALSE",
    "TRUE",
    "FALSE",
    "FALSE",
    "FALSE",
    "TRUE",
    "TRUE",
    "TRUE"
]

In [ ]:
print(len(texts_3))
print(len(true_labels_3))

In [ ]:
# Batch 4 with 11 texts
texts_4 = [
    "Clarendon Holdings, Inc., a subsidiary of Enstar Group Limited, agreed to purchase Clarendon National Insurance Company from Clarendon Insurance Group, Inc. for approximately $200 million. The purchase price is subject to adjustment based on the acquired company's consolidated surplus as of December 31, 2010. The transaction is expected to be financed partially through a bank loan facility and cash on hand. The completion of the deal is subject to regulatory approval and customary closing conditions, with an expected closing date in the second quarter of 2011. The acquisition agreement includes guarantees from Hannover Finance, Inc. and Enstar Group Limited. The report contains forward-looking statements cautioning investors about potential risks and uncertainties that could impact the completion of the transaction, including failure to obtain regulatory approvals or satisfy closing conditions. Enstar Group Limited is not obligated to update any forward-looking statements unless required by law.",
    "    On April 15, 2013, Concho Resources Inc. (the  Company ) entered into an Eleventh Amendment (the  Eleventh Amendment ) to its Amended and Restated  Credit Agreement, dated as of July 31, 2008, with the lenders party thereto and JPMorgan Chase Bank, N.A., as administrative agent, as amended (the  Amended and Restated Credit Agreement ).  Prior to effectiveness of the Eleventh Amendment, the Company was prohibited from issuing unsecured notes in excess of $4.0 billion, and  any issuance of unsecured notes after November 2, 2013 would have resulted in a reduction in the Companys borrowing base equal to the lesser of (i) $300 for every $1,000 of aggregate principal amount of unsecured senior notes issued  by the Company or (ii) such other amount, if any, as determined prior to the issuance of such notes by lenders holding at least two-thirds of the outstanding credit exposure and unused commitments under the Amended and Restated Credit  Agreement. The Eleventh Amendment removes the $4.0 billion limit on the Companys ability to issue unsecured notes as well as the provision requiring that any issuance of unsecured notes after November 2, 2013 would result in a reduction  of the borrowing base under the Amended and Restated Credit Agreement.    The Eleventh Amendment also allows the Company to make  restricted payments in an aggregate amount up to $250.0 million plus (i) 50% of the Companys consolidated net income (or, in the case of a consolidated net loss, minus 100% of such deficit) beginning January 1, 2013 and (ii) 66  2/3% of the aggregate net cash proceeds received by the Company from any issuances of shares of common stock on or after January 1, 2013.    Before entering into the Eleventh Amendment, the Company was permitted to enter into commodities hedging arrangements with aggregate notional volume for any given month that did not exceed (i) 100%  of the Companys expected production from proven producing reserves for any given month during the succeeding two-year period and (ii) 85% of the Companys expected production from proven producing reserves for any given month during  the remaining three years of the succeeding five-year period. The Eleventh Amendment permits the Company to enter into commodities hedging arrangements with aggregate notional volume for any given month that do not exceed (i) 90% of the  Companys forecasted production of oil and natural gas reserves, based on internal engineering reports, for any given month during the succeeding one-year period and (ii) 80% of the Companys forecasted production of oil and natural  gas reserves, based on internal engineering reports, for any given month during the remaining four years of the succeeding five-year period. In addition, the Eleventh Amendment provides that the Company may enter into commodities hedging  arrangements covering up to 70% of its forecasted production of oil and natural gas reserves, based on internal engineering reports, with respect to estimated reserves to be acquired in connection with certain acquisitions.  Finally, the Eleventh Amendment reaffirms the Companys current borrowing base of $3.0 billion and maintains the lenders  aggregate commitments of $2.5 billion under the Amended and Restated Credit Agreement.    The foregoing description of the  Eleventh Amendment is a summary only and is qualified in its entirety by reference to the complete text of the Eleventh Amendment, a copy of which is attached as Exhibit 10.1 to this Current Report on Form 8-K and incorporated herein by reference.",
    " On December 29, 2017, the Company, through its subsidiary Laguna Life Holdings SARL (""Laguna""), completed the previously announced sale of its subsidiary, Pavonia Holdings (US), Inc. (""Pavonia""), to Southland National Holdings, Inc. (""Southland""), a Delaware corporation and a subsidiary of Global Bankers Insurance Group, LLC.  The aggregate purchase price was $120 million. The Company used the proceeds to make repayments under its revolving credit facility. Pavonia owns Pavonia Life Insurance Company of Michigan (""PLIC MI"") and Enstar Life (US), Inc.  Pursuant to the amended stock purchase agreement between Laguna and Southland, which partially restructured the transaction, Southland will acquire Pavonia Life Insurance Company of New York (""PLIC NY"") for $13.1 million in a second closing that is expected to occur in the first or second quarter of 2018, subject to regulatory approval.  The additional purchase price represents the cash consideration paid by Laguna to PLIC MI when Laguna acquired PLIC NY from PLIC MI in the restructuring of the transaction.",
    "  On April 25, 2011, Concho Resources Inc. (the  Company ) has entered into a Sixth Amendment  (the  Sixth Amendment ) to its Amended and Restated Credit Agreement, with the lenders party  thereto and JPMorgan Chase Bank, N.A., as administrative agent (the  Amended and Restated Credit  Agreement ).  The Sixth Amendment extends the maturity of the Companys credit facility under the Amended  and Restated Credit Agreement from July 31, 2013 to April 25, 2016. In addition, the Sixth  Amendment increases the borrowing base under the Amended and Restated Credit Agreement from $2.0  billion to $2.5 billion. In connection with the borrowing base increase, the Sixth Amendment  provides the Company with the ability to increase the maximum size of the credit facility from the  existing $2.0 billion up to a maximum of $2.5 billion from time to time, subject to the lenders  agreeing to provide any increased commitments. The Sixth Amendment also modifies the interest rate  spreads for the Companys borrowings under the Amended and Restated Credit Agreement as follows:  Type  Prior Range  New Range  Eurodollar borrowings  2.00% to 3.00%  1.50% to 2.50%  ABR borrowings  1.125% to 2.125%  0.50% to 1.50%  Unused commitment fee  0.50  %  0.375% to 0.50%  The Sixth Amendment provides for certain modifications with respect to acquisitions by  the Company. Specifically, the Sixth Amendment: (i) modifies the calculation of the Companys  Consolidated Leverage Ratio by utilizing the Companys Consolidated EBITDAX after giving pro forma  effect to acquisitions or divestitures in excess of $25 million; and (ii) provides the Company with  the flexibility to enter into additional hedges in contemplation of acquisitions, subject to a  limit equal to the lesser of (x) 150% of the hedged volumes allowed prior to the acquisition and  (y) the maximum volume allowed to be hedged after giving pro forma effect to the contemplated  acquisition.  Finally, the Sixth Amendment increases the limit on the Companys aggregate outstanding senior  unsecured notes from $900 million to $1.9 billion and allows the Company to issue additional senior  unsecured notes prior to May 1, 2012 without any reduction in the borrowing base as a result of the  issuance of such notes. After May 1, 2012, any issuance of senior unsecured notes will result in a  reduction in the borrowing base equal to the lesser of (i) $0.30 for every dollar of senior  unsecured notes issued by the Company or (ii) such other amount as determined by two-thirds of the  lenders upon any such issuance.  The foregoing description of the Sixth Amendment is a summary only and is qualified in its  entirety by reference to the complete text of the Sixth Amendment, a copy of which is attached as  Exhibit 10.1 to this Current Report on Form 8-K and incorporated herein by reference.",
    "  On August 18, 2017, Innoviva, Inc. (Innoviva or the Company) entered into a Credit Agreement (the Credit Agreement) with Morgan Stanley Senior Funding, Inc. (MSSF), as administrative agent and collateral agent, the other agents party thereto and the lenders referred to therein (collectively, the Lenders). Pursuant to the Credit Agreement, the Lenders have provided $250,000,000 in senior secured first lien term loans (the Term Loans). The proceeds from the Term Loans will be used by the Company to  redeem in full the LABA PhaRMA sm  9.0% Fixed Rate Term Notes due 2029 (the 2029 Notes), issued pursuant to the Amended and Restated Indenture, dated as of August 3, 2016 (as further amended, supplemented or otherwise modified from time to time prior to its termination hereunder, the LABA Indenture), by and between LABA Royalty Sub LLC, a Delaware limited liability company and wholly owned subsidiary of Innoviva (LABA Royalty Sub), as the issuer, and U.S. Bank National Association (U.S. Bank), as trustee of the 2029 Notes (in such capacity, the LABA Trustee), and to pay related fees and expenses.  The Term Loans bear interest based on a fluctuating rate of interest measured, at Innovivas option, at an alternative base rate equal to the highest of the (a) prime rate, (b)  1/2 of 1% per annum above the federal funds effective rate, (c) LIBOR for an interests period of one month plus 1.00% and (d) 2.00% (the Base Rate) or LIBOR, plus a margin. The margin for Base Rate borrowings under the Credit Agreement is 3.50% and for LIBOR borrowings under the Credit Agreement is 4.50%. Borrowings based on LIBOR under the Credit Agreement will be subject to a 1.00% LIBOR floor. Interest for Base Rate-based loans is calculated on the basis of a 365/366-day year and interest for LIBOR-based loans is calculated on the basis of a 360-day year.  Subject to certain customary exceptions, all obligations of Innoviva under the Credit Agreement are unconditionally guaranteed by Innovivas wholly owned subsidiary, Advanced Medicine East, Inc., and will be required to be guaranteed by each of Innovivas subsequently acquired or organized direct and indirect restricted wholly-owned domestic subsidiaries whose assets or net revenues exceed 5% of the consolidated assets or net revenues, as the case may be, of Innoviva and its restricted subsidiaries (the Guarantors). Other domestic restricted subsidiaries, subject to certain customary exceptions, will be required to become Guarantors to the extent that domestic restricted subsidiaries excluded from such guarantee obligation represent, in the aggregate, more than 10% of the consolidated assets and more than 10% of the consolidated net revenues of Innoviva.  The obligations under the Credit Agreement are Innovivas and the Guarantors senior secured obligations, collateralized by a lien on substantially all of Innovivas and the Guarantors personal property and material real property assets (if any) (subject in each case to certain customary exceptions).  The Term Loans may be prepaid, in whole or in part, without premium or penalty, except that (i) prepayments financed by the incurrence of lower-yielding indebtedness (other than in connection with certain change-of-control or transformative acquisitions) within six months from August 18, 2017 will be subject to a prepayment premium of 1.0% of the principal amount prepaid and (ii) lenders will be compensated for LIBOR breakage costs (if any).  The Term Loans mature five years following the effective date. The Credit Agreement requires quarterly principal payments equal to 2.50% of the original principal amount of the Term Loans.  Additionally, the Credit Agreement stipulates an annual principal payment of a percentage of Excess Cash Flow (as defined in the Credit Agreement) (ECF) to repay the Term Loans. The first ECF application date will be measured as of the end of fiscal year 2018 and the ECF percentage is scheduled to be 50% if the consolidated first lien secured leverage ratio of the Company as of the last day of the applicable fiscal year is greater than 1.55 to 1.00, 25% if the consolidated first lien secured leverage ratio of the Company as of the last day of the applicable fiscal year is less than or equal to 1.55 to 1.00 but greater than 0.55 to 1.00 and 0% otherwise.  2  The Term Loans are also subject to prepayment from (i) the net cash proceeds of certain debt incurred or issued by Innoviva and its restricted subsidiaries and (ii) the net cash proceeds received by Innoviva or its restricted subsidiaries from certain assets sales and recovery events, subject to certain reinvestment rights.  Innoviva can request, at any time and from time to time, subject to the satisfaction of customary conditions, the establishment of one or more incremental term loan facilities with commitments in an aggregate amount not to exceed, (i) (x) in the case of incremental term loans secured on a pari passu basis with the Term Loans, an amount such that the consolidated first lien secured leverage ratio of the Company calculated on a pro forma basis would not be greater than 2.00 to 1.00 or (y) in the case of incremental term loans secured on a junior basis to the Term Loans or unsecured, an amount such that the consolidated leverage ratio of the Company calculated on a pro forma basis would not be greater than 3.56 to 1.00, plus (ii) the greater of (x) $75,000,000 and (y) 45% of the prior four quarters consolidated EBITDA of the Company, plus (iii) the amount of any voluntary prepayments of the Term Loans other  than such voluntary prepayments financed with the proceeds of long term indebtedness. Lenders are not obligated to commit to provide any such incremental term loan facility requested by Innoviva.  The Credit Agreement also contains restrictive covenants that limit, among other things, Innovivas ability and that of Innovivas restricted subsidiaries to incur additional indebtedness or issue certain preferred equity, pay dividends or make other distributions or other restricted payments, make certain investments, create restrictions on distributions from subsidiaries (including prepayments of junior indebtedness), amend the terms of certain other indebtedness, create liens on certain assets, sell certain assets, consolidate, merge, liquidate, dissolve, sell or otherwise dispose of all or substantially all of its assets, enter into certain hedging arrangements, alter its line of business and enter into certain transactions with affiliates. The Credit Agreement also contains customary events of default, including upon the failure to make timely payments under the Credit Agreement or other material indebtedness, the failure to satisfy certain covenants, the occurrence of a change of control and specified events of bankruptcy and insolvency.  The Credit Agreement provides for customary events of default, including: non-payment of principal, interest or other amounts payable under the Credit Agreement or related loan document; violation of covenants; material inaccuracy of representations or warranties; cross default to other material indebtedness; certain bankruptcy and insolvency events; certain ERISA events; material invalidity of guarantees or security interests; failure of the obligations of the Company under the Credit Agreement and related loan documents to constitute senior indebtedness with respect to the Companys 2.125% Convertible Subordinated Notes due 2023; material judgments; and change of control.  In connection with the redemption in full of the 2029 Notes and the execution of the LABA Termination Agreement (as defined below), on August 18, 2017, Innoviva and LABA Royalty Sub entered into an assignment agreement (the Assignment Agreement), pursuant to which LABA Royalty Sub transferred and assigned to Innoviva all assets of LABA Royalty Sub, including (i) the right, title and interest of LABA Royalty Sub, in all of the Collateral (as defined in the LABA Indenture) and proceeds thereof and (ii) the right, title and interest of LABA Royalty Sub, in the accounts and other assets and property of LABA Royalty Sub that are not part of the Collateral.  The above descriptions of the Credit Agreement and the Assignment Agreement are a summary only and are qualified in their entirety by reference to the Credit Agreement and the Assignment Agreement, which are attached hereto as Exhibits 10.1 and 10.2, respectively, and are incorporated herein by reference.",
    " Convertible Note Offering  On August 7, 2017, Innoviva, Inc. (Innoviva or the Company) completed its previously announced offering of $175.0 million aggregate principal amount of its 2.50% Convertible Senior Notes due 2025 (the Notes). The Notes were sold in a private placement under a purchase agreement, dated as of August 1, 2017, entered into by and among Innoviva and Morgan Stanley & Co. LLC and Deutsche Bank Securities Inc., as initial purchasers (collectively, the Initial Purchasers), for resale to qualified institutional buyers pursuant to Rule 144A under the Securities Act of 1933, as amended (the Securities Act). Innoviva also granted the Initial Purchasers a right to purchase, within a 30-day period, up to an additional $17.5 million principal amount of additional Notes on the same terms and conditions, which the Initial Purchasers exercised in full on August 4, 2017 and which additional purchase was also completed on August 7, 2017.  The net proceeds from the sale of the $192.5 million aggregate principal amount of Notes were approximately $187.3 million after deducting the Initial Purchasers discounts and commissions and Innovivas estimated offering expenses. Innoviva used approximately $17.5 million of the net proceeds from this offering to repurchase 1,317,771 shares of its common stock, at a price per share of $13.28, concurrently with the pricing of the offering of the Notes in privately negotiated transactions effected through one of the Initial Purchasers or its affiliate, as Innovivas agent. Innoviva currently intends to use the remainder of the net proceeds of this offering to repay a portion of the principal outstanding under the Companys LABA PhaRMA SM  9.0% Fixed Rate Term Notes due 2029 on the next interest payment date of August 15, 2017.  Indenture  On August 7, 2017, Innoviva entered into an indenture (the Indenture) with respect to the Notes with The Bank of New York Mellon Trust Company, N.A., as trustee (the Trustee). Under the Indenture, the Notes will be senior unsecured obligations of Innoviva and bear interest at a rate of 2.50% per year, payable semi-annually in arrears on February 15 and August 15 of each year, beginning on February 15, 2018. The Notes will mature on August 15, 2025, unless earlier repurchased or converted.  The Notes are convertible into shares of Innovivas common stock, cash or a combination of shares of Innovivas common stock and cash, at Innovivas election, based on an initial conversion rate of 57.9240 shares per $1,000 principal amount of Notes (which is equivalent to an initial conversion price of approximately $17.26 per share), subject to adjustment upon the occurrence of certain events. The initial conversion price represents a premium of approximately 30% to the $13.28 per share closing price of Innovivas common stock on August 1, 2017.  Prior to February 15, 2025, the Notes will be convertible at the option of the holders only upon the occurrence of specified events and during certain periods, and will be convertible on or after February 15, 2025 at any time until the close of business on the second scheduled trading day immediately preceding the maturity date of the Notes.  If certain corporate events that constitute a make-whole fundamental change as set forth in the Indenture occur prior to the maturity date, the conversion rate may, in certain circumstances, be increased for a holder who elects to convert its Notes in connection with such corporate event.  The conversion rate is subject to customary anti-dilution provisions.  The Notes are not redeemable prior to maturity, and no sinking fund is provided for the Notes. If Innoviva undergoes a fundamental change, as defined in the Indenture, subject to certain conditions, holders may require Innoviva to purchase for cash all or any portion of their Notes. The fundamental change purchase price will be 100% of the principal amount of the Notes to be purchased plus any accrued and unpaid interest to, but excluding, the fundamental change purchase date.  The Indenture contains customary terms and covenants, including that upon certain events of default occurring and continuing, either the Trustee or the holders of at least 25% of the aggregate principal amount of the outstanding Notes may declare 100% of the principal of, and accrued and unpaid interest, if any, on, all the Notes to be due and payable immediately.  2  The above description of the Indenture and the Notes is a summary only and is qualified in its entirety by reference to the Indenture (and the Form of Note included therein), which is attached hereto as Exhibit 4.1 and is incorporated herein by reference.",
    " On December 9, 2019, Enstar Group Limited and certain of its subsidiaries, as guarantors, and its wholly-owned subsidiary Cavello Bay Reinsurance Limited (""Cavello Bay""), as borrower, entered into an amendment (the ""First Amendment"") to the unsecured $600 million letter of credit facility agreement dated August 5, 2019 with the lenders party thereto and National Australia Bank Limited, London Branch (""NAB"") as administrative agent, National Australia Bank Limited as several L/C issuing bank, and NAB and The Bank of Nova Scotia as joint bookrunners, joint lead arrangers and syndication agents (the ""Facility Agreement"" and, as amended by the First Amendment, the ""Amended Facility Agreement""). The First Amendment increases the total commitments under the Facility Agreement from $600 million to $760 million. In addition, pursuant to the Amended Facility Agreement, Cavello Bay has the option to increase the commitments in an aggregate amount up to $40 million from the existing lenders or through the addition of one or more new lenders, subject to the terms of the Amended Facility Agreement. Cavello Bay intends to use the increased commitments to provide letters of credit in support of certain of its reinsurance obligations in the ordinary course of business. The foregoing description of the First Amendment is qualified by reference to the full text of the First Amendment filed as Exhibit 10.1 to this Current Report on Form 8-K, which is incorporated herein by reference.",
    "  As partial  consideration for the Acquisition, the Company has agreed to issue the Note to Pitch in the  aggregate principal amount of $150 million. The note will (i) bear interest at the rate of 8% per  year, payable semi-annually in arrears, (ii) be payable as to principal in a lump sum on the eighth  anniversary of the closing of the Acquisition and (iii) provide for the Companys option to prepay  the note, together with accrued interest thereon, from time to time, in whole or in part, without  penalty or premium.  In addition, the information concerning the Commitments set forth under Item 1.01 above is  incorporated herein by reference.",
    "Item 2.03 of the Current Report on Form 8-K incorporates provisions from Item 1.01 related to the Restated Credit Agreement, Trident put rights for its interests in Northshore and Bayshore.",
    "  On December 21, 2012, AML Acquisition, Corp. (AML Acquisition), an indirect, wholly-owned subsidiary of Enstar Group  Limited (the Company), as borrower, entered into a Term Facility Agreement with National Australia Bank Limited and Barclays Bank PLC (the SeaBright Facility). The SeaBright Facility provides for a four-year term loan  facility pursuant to which AML Acquisition is permitted to borrow up to an aggregate of $111.0 million, which will be available to fund up to 50% of the purchase price for the Companys previously announced agreement to acquire SeaBright  Holdings, Inc. (SeaBright). The Company plans to acquire SeaBright by way of a merger of AML Acquisition with and into SeaBright (the Merger), with SeaBright surviving the Merger as an indirect, wholly-owned subsidiary of the  Company. As of the date of this Current Report on Form 8-K, AML Acquisition has not borrowed any of the amount available under the SeaBright Facility. Consummation of the Merger is subject to customary closing conditions, including the receipt of  certain regulatory approvals. The Merger is currently expected to close in the first quarter of 2013.    The SeaBright Facility  is currently secured by a security interest in all of the assets of AML Acquisition, as well as a pledge of the stock of AML Acquisition by its sole stockholder. Following completion of the Merger, SeaBright (as the survivor of the Merger) would  become the borrower under the SeaBright Facility and the facility would be secured by a security interest in all of the assets of SeaBright, a pledge of the stock of SeaBright by its sole stockholder, a pledge of the stock of SeaBright Insurance  Company, Paladin Managed Care Services, Inc., and PointSure Insurance Services, Inc. (which are wholly-owned subsidiaries of SeaBright) by SeaBright, and a security interest in all of the assets of Paladin Managed Care Services, Inc. and PointSure  Insurance Services, Inc.    From the date of the SeaBright Facility until the date that is one month after the completion of the  Merger, the undrawn and uncancelled amount of the SeaBright Facility will incur a fee of 1% per annum. Interest on amounts borrowed will be payable at the end of each interest period chosen by the borrower or, at the latest, each six months.  The interest rate will be LIBOR plus 2.75% for the first 18 months from the date on which the loan under the SeaBright Facility is made, and will increase to LIBOR plus 3.50% thereafter; the interest rate is subject to increase by an incremental  amount tied to certain regulatory costs, if any, that may be incurred by the lenders. The SeaBright Facility imposes various financial and business covenants on AML Acquisition (and, following completion of the Merger, SeaBright), including  limitations on mergers and consolidations, acquisitions, indebtedness and guarantees, restrictions as to dispositions of stock and assets (except for certain permitted dispositions), restrictions on dividends, and limitations on liens.  During the existence of any payment default, the interest rate would be increased by 1.0%. During the existence of any event of default  (as specified in the SeaBright Facility), the lenders may cancel their commitments, declare all or a portion of outstanding amounts immediately due and payable, declare all or a portion of borrowed amounts payable upon demand, or proceed against the  security. The SeaBright Facility terminates and all amounts borrowed must be repaid on the fourth anniversary of the date the term loan is made.    Cautionary Statement Regarding Forward-Looking Statements    This Current Report on Form 8-K  contains certain forward-looking statements within the meaning of the Private Securities Litigation Reform Act of 1995. These statements include statements regarding the intent, belief or current expectations of the Company and its management team.  Investors are cautioned that any such forward-looking statements are not guarantees of future performance and involve risks and uncertainties, and that actual results may differ materially from those projected in the forward-looking statements as a  result of various factors. In particular, the Company may not be able to complete the Merger due to a number of factors, including but not limited to a failure to obtain governmental and regulatory approvals or to satisfy other closing conditions.  The foregoing list of important factors is not exhaustive. Other important risk factors regarding the Company may be found under the heading  Risk Factors in the Companys Form 10-K for the year ended December 31, 2011, and are incorporated herein by reference. Furthermore, the Company undertakes no obligation to update any written or oral forward-looking statements  or publicly announce any updates or revisions to any of the forward-looking statements contained herein, to reflect any change in its expectations with regard thereto or any change in events, conditions, circumstances or assumptions underlying such  statements, except as required by law.",
    "In anticipation of consummating the transactions described in Item 1.01 of this Current Report on Form 8-K, on July 15, 2021, Enstar borrowed $300 million under the Revolving Credit Facility, the proceeds of which will be used to fund in part the purchase under the EGL Agreement and pay costs and expenses associated therewith. Following the draw down, Enstar has approximately $125 million of undrawn borrowing capacity under the Revolving Credit Facility. A summary of the material terms of the Revolving Credit Facility is included in Enstar's Annual Report on  Form 10-K  for the year ended December 31, 2020 filed with the SEC on March 1, 2021 and is incorporated herein by reference"
   ]

true_labels_4 = [
    "FALSE",
    "TRUE",
    "FALSE",
    "TRUE",
    "TRUE",
    "TRUE",
    "TRUE",
    "TRUE",
    "FALSE",
    "TRUE",
    "TRUE"
]

In [ ]:
print(len(texts_4))
print(len(true_labels_4))

In [ ]:
# Concatenate
texts = texts_1 + texts_2 + texts_3 + texts_4
true_labels = true_labels_1 + true_labels_2 + true_labels_3 + true_labels_4

import time
predictions = []
for text in texts:
    pred = run_mistral_clean(text)
    predictions.append(pred)
    time.sleep(2)

In [ ]:
len(predictions)

In [ ]:
# Sanity check
for index, elem in enumerate(predictions):
  text = texts[index]
  print("Text: ", text)
  print(elem)
  print("=======")
# Study responses and look back at examples in prompt

Metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print(len(true_labels), len(predictions))

# Make sure all labels are strings
true_labels = [str(label).strip().upper() for label in true_labels]
predictions = [str(pred).strip().upper() for pred in predictions]

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='macro',zero_division=0)
recall = recall_score(true_labels, predictions, average='macro',zero_division=0)
f1 = f1_score(true_labels, predictions, average='macro',zero_division=0)

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predictions)

# Plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['False', 'True'])
disp.plot(cmap='Blues')
plt.title('Mistral Confusion Matrix: Summarized Validation Dataset')
plt.tight_layout()
plt.show()

In [ ]:
# Combine all parts into one DataFrame
new_summarized = pd.DataFrame({
    'text': texts,
    'true': true_labels,
    'preds': predictions
})

from wordcloud import WordCloud
import random
custom_colors = ["#4C69B2", "#FF4949", "#FF7A78", "#FF9D35", "#6389EB", "#FFC53A"]
def random_color_func(word=None, font_size=None, position=None, orientation=None, font_path=None, random_state=None):
    return random.choice(custom_colors)

# Convert string labels like "TRUE"/"FALSE" to 1/0
label_map = {"TRUE": 1, "FALSE": 0}

new_summarized["true"] = new_summarized["true"].map(label_map)
new_summarized["preds"] = new_summarized["preds"].map(label_map)

# Subsets
fp = new_summarized[(new_summarized["preds"] == 1) & (new_summarized["true"] == 0)]
fn = new_summarized[(new_summarized["preds"] == 0) & (new_summarized["true"] == 1)]

# Combine all FP/FN texts
fp_text = " ".join(fp['text'])
fn_text = " ".join(fn['text'])

# Generate word clouds with custom color
wordcloud_fp = WordCloud(width=800, height=400, background_color='white', color_func=random_color_func).generate(fp_text)
wordcloud_fn = WordCloud(width=800, height=400, background_color='white', color_func=random_color_func).generate(fn_text)

# Plot
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(wordcloud_fp, interpolation='bilinear')
plt.title('False Positives')
plt.axis('off')

In [ ]:
# Save as CSV
output_path = '/content/gdrive/MyDrive/DSSI 2025/Group 1: DSSI Summer 2025/Data/Prompt Engineering Datasets/mistral_summarized.csv'
new_summarized.to_csv(output_path, index=False)